In [ ]:
# Importing the libraries
import numpy as np 
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

# Rest of your code stays exactly the same
import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import CSS4_COLORS, hsv_to_rgb
import matplotlib as mpl
import seaborn as sns
import ast 
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
import plotly.express as px
from datetime import timedelta, datetime
import openai
import time 
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.manifold import TSNE
import hdbscan
from sklearn.preprocessing import StandardScaler
import os
import pickle
import re
from collections import Counter
from tqdm.auto import tqdm
from tqdm import tqdm
import json
from tqdm.notebook import tqdm
import random
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import qualitative, sequential
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.colors

# Show ALL columns
pd.set_option('display.max_columns', None)

# set openai api key
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4.1" 


In [ ]:
#Load Dataset
sku_data = pd.read_parquet('sku_data.parquet')

#Inspect Datasets: Overview, Data Types, and Missing Values
def inspect_dataset(df):
    print(f"\n{df} Dataset Overview:")
    print("\nHEAD:\n", df.head())
    print("\nINFO:\n")
    print(df.info())
    print("\nMISSING VALUES:\n", df.isnull().sum())
    print("\n" + "-"*80 + "\n")

# Run inspections
#df = sku_data.sample(n=20, random_state=None)

# Convert date columns to datetime, handling missing values
sku_data['created_date'] = pd.to_datetime(sku_data['created_date'], errors='coerce')
sku_data['updated_date'] = pd.to_datetime(sku_data['updated_date'], errors='coerce')

# Calculate the earliest and latest dates in the 'created_date' column
earliest_created_date = sku_data['created_date'].min()
latest_created_date = sku_data['created_date'].max()

print("Earliest created_date:", earliest_created_date)
print("Latest created_date:", latest_created_date)

# Calculate the earliest and latest dates in the 'updated_date' column
earliest_updated_date = sku_data['updated_date'].min()
latest_updated_date = sku_data['updated_date'].max()

print("Earliest updated_date:", earliest_updated_date)
print("Latest updated_date:", latest_updated_date)

# Set now to the latest updated_date
now = latest_updated_date

inspect_dataset(sku_data)
sku_data


In [ ]:
# Add cost_per_unit column with value 13.73 for all SKUs
sku_data['cost_per_unit'] = 13.73

sku_data

In [ ]:
# Identify rows with empty historical_inventory, historical_sales, and historical_received
empty_inventory_sales_received = sku_data[
    (sku_data['historical_inventory'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x))) & 
    (sku_data['historical_sales'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x))) & 
    (sku_data['historical_received'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x)))
]

# Print the rows that will be removed
print("Rows with empty historical_inventory, historical_sales, and historical_received:")
print(empty_inventory_sales_received[['sku', 'name']])

# Remove these rows from sku_data
sku_data = sku_data[~((sku_data['historical_inventory'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x))) & 
                      (sku_data['historical_sales'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x))) & 
                      (sku_data['historical_received'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x))))]

empty_inventory = sku_data[
    sku_data['historical_inventory'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x))
]

# Print the rows that will be removed for historical_inventory
print("\n\nRows with empty historical_inventory:")
print(empty_inventory[['sku', 'name']])
print("\n\nNumber of rows with empty historical_inventory:", empty_inventory.shape[0])


empty_sales = sku_data[
    sku_data['historical_sales'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x))
]

# Print the rows that will be removed for historical_sales
print("\n\nRows with empty historical_sales:")
print(empty_sales[['sku', 'name']])
print("\n\nNumber of rows with empty historical_sales:", empty_sales.shape[0])
empty_received = sku_data[
    sku_data['historical_received'].apply(lambda x: len(x) == 0 if isinstance(x, (list, np.ndarray)) else pd.isnull(x))
]

# Print the rows that will be removed for historical_received
print("\n\nRows with empty historical_received:")
print(empty_received[['sku', 'name']])
print("\n\nNumber of rows with empty historical_received:", empty_received.shape[0])

In [ ]:
sku_data = sku_data.reset_index(drop=True)
inspect_dataset(sku_data)

In [ ]:
# added inventory collums 
# current_inventory, inventory_30d_ago, inventory_90d_ago, inventory_180d_ago, inventory_360d_ago, initial_inventory

def get_current_inventory(historical_inventory):
    """Get the most recent inventory value"""
    # Handle both empty lists and empty arrays
    if isinstance(historical_inventory, (list, np.ndarray)) and len(historical_inventory) == 0:
        return np.nan
    # Handle pandas Series or other array-like objects
    try:
        if len(historical_inventory) == 0:
            return np.nan
    except:
        return np.nan
    
    value = historical_inventory[-1]['inventory']
    return float(value) if not pd.isna(value) else np.nan

def get_inventory_before_date(historical_inventory, target_date):
    """Find inventory closest to but before a target date"""
    # Handle empty data
    if isinstance(historical_inventory, (list, np.ndarray)) and len(historical_inventory) == 0:
        return np.nan
    try:
        if len(historical_inventory) == 0:
            return np.nan
    except:
        return np.nan
    
    # Convert target_date to pandas Timestamp for consistent comparison
    if hasattr(target_date, 'date') and not hasattr(target_date, 'hour'):
        # It's a date object, convert to pandas Timestamp
        target_date = pd.Timestamp(target_date)
    elif not isinstance(target_date, pd.Timestamp):
        target_date = pd.Timestamp(target_date)
    
    # Make target_date timezone-naive if it has timezone info
    if hasattr(target_date, 'tzinfo') and target_date.tzinfo is not None:
        target_date = target_date.tz_localize(None)
    
    # Filter entries before or equal to target_date
    filtered = []
    for entry in historical_inventory:
        entry_date = entry['date']
        
        # Convert string dates to pandas Timestamp
        if isinstance(entry_date, str):
            try:
                entry_date = pd.Timestamp(entry_date)
            except:
                continue  # Skip entries with invalid date strings
        
        # Convert date objects to pandas Timestamp for comparison
        if hasattr(entry_date, 'date') and not hasattr(entry_date, 'hour'):
            # It's a date object, convert to pandas Timestamp
            entry_date = pd.Timestamp(entry_date)
        elif not isinstance(entry_date, pd.Timestamp):
            entry_date = pd.Timestamp(entry_date)
        
        # Make entry_date timezone-naive if it has timezone info
        if hasattr(entry_date, 'tzinfo') and entry_date.tzinfo is not None:
            entry_date = entry_date.tz_localize(None)
        
        if entry_date <= target_date:
            filtered.append(entry)
    
    if len(filtered) == 0:
        return np.nan
    
    # Return inventory of the last entry before or on target_date
    value = filtered[-1]['inventory']
    return float(value) if not pd.isna(value) else np.nan

def get_inventory_days_ago(historical_inventory, reference_date, days_ago):
    """Calculate inventory N days ago from reference date"""
    target_date = reference_date - timedelta(days=days_ago)
    return get_inventory_before_date(historical_inventory, target_date)

def get_initial_inventory(historical_inventory):
    """Calculate initial inventory by going back one step from first entry"""
    # Handle empty data
    if isinstance(historical_inventory, (list, np.ndarray)) and len(historical_inventory) == 0:
        return np.nan
    try:
        if len(historical_inventory) == 0:
            return np.nan
    except:
        return np.nan
    
    first_entry = historical_inventory[0]
    # Initial inventory = first recorded inventory - quantity_change
    # This gives us the inventory before the first transaction
    initial_inventory = first_entry['inventory'] - first_entry['net_change']
    return float(initial_inventory) if not pd.isna(initial_inventory) else np.nan

# Calculate days_on_market first
sku_data['days_on_market'] = (sku_data['updated_date'] - sku_data['created_date']).dt.days

# Apply these functions to your dataframe
sku_data['current_inventory'] = sku_data['historical_inventory'].apply(get_current_inventory)
sku_data['inventory_30d_ago'] = sku_data['historical_inventory'].apply(
    lambda x: get_inventory_days_ago(x, now, 30)
)
sku_data['inventory_90d_ago'] = sku_data['historical_inventory'].apply(
    lambda x: get_inventory_days_ago(x, now, 90)
)
sku_data['inventory_180d_ago'] = sku_data['historical_inventory'].apply(
    lambda x: get_inventory_days_ago(x, now, 180)
)
sku_data['inventory_360d_ago'] = sku_data['historical_inventory'].apply(
    lambda x: get_inventory_days_ago(x, now, 360)
)
sku_data['initial_inventory'] = sku_data['historical_inventory'].apply(get_initial_inventory)

# Test if target dates are before created dates but still have non-null values
invalid_skus_30d = []
invalid_skus_90d = []
invalid_skus_180d = []
invalid_skus_360d = []

for idx, row in sku_data.iterrows():
    target_date_30d = now - timedelta(days=30)
    target_date_90d = now - timedelta(days=90)
    target_date_180d = now - timedelta(days=180)
    target_date_360d = now - timedelta(days=360)
    
    if target_date_30d < row['created_date'] and not pd.isna(row['inventory_30d_ago']):
        invalid_skus_30d.append(row['sku'])
    if target_date_90d < row['created_date'] and not pd.isna(row['inventory_90d_ago']):
        invalid_skus_90d.append(row['sku'])
    if target_date_180d < row['created_date'] and not pd.isna(row['inventory_180d_ago']):
        invalid_skus_180d.append(row['sku'])
    if target_date_360d < row['created_date'] and not pd.isna(row['inventory_360d_ago']):
        invalid_skus_360d.append(row['sku'])

print("SKUs with invalid 30d inventory values:", invalid_skus_30d)
print("SKUs with invalid 90d inventory values:", invalid_skus_90d)
print("SKUs with invalid 180d inventory values:", invalid_skus_180d)
print("SKUs with invalid 360d inventory values:", invalid_skus_360d)

# Reorder columns to place days_on_market after updated_date
cols = list(sku_data.columns)
updated_date_idx = cols.index('updated_date')
days_on_market_idx = cols.index('days_on_market')

# Remove days_on_market from current position
cols.pop(days_on_market_idx)
# Insert it right after updated_date
cols.insert(updated_date_idx + 1, 'days_on_market')

# Reorder the dataframe
sku_data = sku_data[cols]

sku_data

In [ ]:
# Drop specific SKUs: LGBOX, MDBOX, SMBOX
skus_to_drop = ['LGBOX', 'MDBOX', 'SMBOX']
# Show rows before dropping them
rows_to_drop = sku_data[sku_data['sku'].isin(skus_to_drop)]
print(f"Rows to be dropped ({len(rows_to_drop)} SKUs):")
print(rows_to_drop)

sku_data = sku_data[~sku_data['sku'].isin(skus_to_drop)]
print(f"Dropped {len(skus_to_drop)} SKUs: {skus_to_drop}")

# Find all rows with negative initial inventory
negative_initial_inventory = sku_data[sku_data['initial_inventory'] < -100]
print(f"Found {len(negative_initial_inventory)} SKUs with negative initial inventory:")
negative_initial_inventory

In [ ]:
# added recived inventory info
#recived_inventory_30d_ago, recived_inventory_90d_ago, recived_inventory_180d_ago, recived_inventory_360d_ago, recived_inventory

def get_total_received_inventory(historical_received):
    """Calculate total received inventory from historical_received list"""
    if isinstance(historical_received, (list, np.ndarray)) and len(historical_received) == 0:
        return 0.0  # Changed np.nan to 0.0
    try:
        if len(historical_received) == 0:
            return 0.0  # Changed np.nan to 0.0
    except:
        return 0.0  # Changed np.nan to 0.0
    
    total_received = 0.0
    for entry in historical_received:
        total_received += float(entry.get('quantity', 0))
    return total_received

def get_received_inventory_in_period(historical_received, reference_date, days_ago):
    """Calculate total received inventory in the period from days_ago to now"""
    if isinstance(historical_received, (list, np.ndarray)) and len(historical_received) == 0:
        return 0.0
    try:
        if len(historical_received) == 0:
            return 0.0
    except:
        return 0.0
    
    # Calculate the start date of the period (days_ago from reference_date)
    start_date = reference_date - timedelta(days=days_ago)
    
    # Convert all dates to date objects for consistent comparison
    if hasattr(reference_date, 'date'):
        reference_date = reference_date.date()
    if hasattr(start_date, 'date'):
        start_date = start_date.date()
    
    total_received = 0.0
    for entry in historical_received:
        entry_date = entry['date']
        # Convert entry date to date object for consistent comparison
        if hasattr(entry_date, 'date'):
            entry_date = entry_date.date()
        
        # Check if entry date is within the period (start_date < entry_date <= reference_date)
        if start_date < entry_date <= reference_date:
            total_received += float(entry.get('quantity', 0))
    
    return total_received

# Create a copy of the dataframe to avoid SettingWithCopyWarning
sku_data = sku_data.copy()

# Apply received inventory functions to your dataframe
sku_data['received_inventory_30d_ago'] = sku_data['historical_received'].apply(
    lambda x: get_received_inventory_in_period(x, now, 30)
).astype('float64')
sku_data['received_inventory_90d_ago'] = sku_data['historical_received'].apply(
    lambda x: get_received_inventory_in_period(x, now, 90)
).astype('float64')
sku_data['received_inventory_180d_ago'] = sku_data['historical_received'].apply(
    lambda x: get_received_inventory_in_period(x, now, 180)
).astype('float64')
sku_data['received_inventory_360d_ago'] = sku_data['historical_received'].apply(
    lambda x: get_received_inventory_in_period(x, now, 360)
).astype('float64')
sku_data['received_inventory'] = sku_data['historical_received'].apply(get_total_received_inventory).astype('float64')

sku_data

In [ ]:
inspect_dataset(sku_data)

In [ ]:
# Added sales info
# total_sales, sales_last_90d, sales_last_180d, sales_last_360d

def get_total_sales(historical_sales):
    """Calculate total quantity sold from historical_sales list"""
    if isinstance(historical_sales, (list, np.ndarray)) and len(historical_sales) == 0:
        return 0
    try:
        if len(historical_sales) == 0:
            return 0
    except:
        return 0
    
    total_sold = 0
    for entry in historical_sales:
        total_sold += entry.get('quantity', 0)
    return total_sold

def get_sales_before_date(historical_sales, target_date):
    """Calculate total sales up to and including target_date"""
    if isinstance(historical_sales, (list, np.ndarray)) and len(historical_sales) == 0:
        return 0
    try:
        if len(historical_sales) == 0:
            return 0
    except:
        return 0
    
    # Convert target_date to date object for consistent comparison
    if hasattr(target_date, 'date'):
        target_date = target_date.date()
    
    total_sold = 0
    for entry in historical_sales:
        entry_date = entry['date']
        # Convert entry date to date object for consistent comparison
        if hasattr(entry_date, 'date'):
            entry_date = entry_date.date()
        
        if entry_date <= target_date:
            total_sold += entry.get('quantity', 0)
    
    return total_sold

def get_sales_days_ago(historical_sales, reference_date, days_ago):
    """Calculate sales N days ago from reference date"""
    target_date = reference_date - timedelta(days=days_ago)
    return get_sales_before_date(historical_sales, target_date)

# Apply sales functions to your dataframe
sku_data['sales_last_30d'] = sku_data['historical_sales'].apply(
    lambda x: get_sales_days_ago(x, now, 0) - get_sales_days_ago(x, now, 30)
)
sku_data['sales_last_90d'] = sku_data['historical_sales'].apply(
    lambda x: get_sales_days_ago(x, now, 0) - get_sales_days_ago(x, now, 90)
)
sku_data['sales_last_180d'] = sku_data['historical_sales'].apply(
    lambda x: get_sales_days_ago(x, now, 0) - get_sales_days_ago(x, now, 180)
)
sku_data['sales_last_360d'] = sku_data['historical_sales'].apply(
    lambda x: get_sales_days_ago(x, now, 0) - get_sales_days_ago(x, now, 360)
)
sku_data['total_sales'] = sku_data['historical_sales'].apply(get_total_sales)

sku_data


In [ ]:
# Added avg_unit_price, min_unit_price, max_unit_price, total_revenue, revenue_last_30d, revenue_last_60d, revenue_last_90d, revenue_last_180d, 
# revenue_last_360d, avg_daily_revenue_30d, avg_daily_revenue_60d, avg_daily_revenue_90d, avg_daily_revenue_180d, avg_daily_revenue_360d


def get_avg_unit_price(historical_sales):
    """Calculate quantity-weighted average unit price from historical_sales list"""
    if isinstance(historical_sales, (list, np.ndarray)) and len(historical_sales) == 0:
        return 0
    try:
        if len(historical_sales) == 0:
            return 0
    except:
        return 0
    
    total_revenue = 0
    total_quantity = 0
    
    for entry in historical_sales:
        unit_price = entry.get('unit_price', None)  # Changed from 'price' to 'unit_price'
        quantity = entry.get('quantity', 0)
        
        if unit_price is not None and quantity > 0:
            total_revenue += unit_price * quantity
            total_quantity += quantity
    
    if total_quantity == 0:
        return 0  
    
    return total_revenue / total_quantity

def get_avg_unit_price_last_n_days(historical_sales, reference_date, days):
    """Calculate quantity-weighted average unit price for the last N days from reference date"""
    if isinstance(historical_sales, (list, np.ndarray)) and len(historical_sales) == 0:
        return 0
    try:
        if len(historical_sales) == 0:
            return 0
    except:
        return 0
    
    # Convert reference_date to date object for consistent comparison
    if hasattr(reference_date, 'date'):
        reference_date = reference_date.date()
    
    cutoff_date = reference_date - timedelta(days=days)
    
    total_revenue = 0
    total_quantity = 0
    
    for entry in historical_sales:
        if isinstance(entry, dict) and 'date' in entry and 'unit_price' in entry and 'quantity' in entry:
            entry_date = entry['date']
            # Convert entry date to date object for consistent comparison
            if hasattr(entry_date, 'date'):
                entry_date = entry_date.date()
            
            if entry_date > cutoff_date and entry_date <= reference_date:
                unit_price = entry.get('unit_price', None)
                quantity = entry.get('quantity', 0)
                
                if unit_price is not None and quantity > 0:
                    total_revenue += unit_price * quantity
                    total_quantity += quantity
    
    if total_quantity == 0:
        return 0
    
    return total_revenue / total_quantity


def get_min_max_unit_price(historical_sales):
    """Extract min and max unit price from historical_sales"""
    if isinstance(historical_sales, (list, np.ndarray)) and len(historical_sales) == 0:
        return {'min_unit_price': 0.0, 'max_unit_price': 0.0}
    try:
        if len(historical_sales) == 0:
            return {'min_unit_price': 0.0, 'max_unit_price': 0.0}
    except:
        return {'min_unit_price': 0.0, 'max_unit_price': 0.0}
    
    unit_prices = []
    for entry in historical_sales:
        if isinstance(entry, dict) and 'unit_price' in entry:
            unit_price = entry.get('unit_price')
            if unit_price is not None and unit_price > 0:
                unit_prices.append(unit_price)
    
    if len(unit_prices) == 0:
        return {'min_unit_price': 0.0, 'max_unit_price': 0.0}
    
    return {
        'min_unit_price': min(unit_prices),
        'max_unit_price': max(unit_prices)
    }

def get_total_revenue(historical_sales):
    """Calculate total revenue from historical_sales"""
    if isinstance(historical_sales, (list, np.ndarray)) and len(historical_sales) == 0:
        return 0
    try:
        if len(historical_sales) == 0:
            return 0
    except:
        return 0
    
    total_revenue = 0
    for entry in historical_sales:
        if isinstance(entry, dict) and 'total_value' in entry:
            total_value = entry.get('total_value', 0)
            if total_value is not None:
                total_revenue += total_value
    
    return total_revenue

def get_revenue_last_n_days(historical_sales, reference_date, days):
    """Calculate total revenue for the last N days from reference date"""
    if isinstance(historical_sales, (list, np.ndarray)) and len(historical_sales) == 0:
        return 0
    try:
        if len(historical_sales) == 0:
            return 0
    except:
        return 0
    
    # Convert reference_date to date object for consistent comparison
    if hasattr(reference_date, 'date'):
        reference_date = reference_date.date()
    
    cutoff_date = reference_date - timedelta(days=days)
    
    revenue_in_period = 0
    for entry in historical_sales:
        if isinstance(entry, dict) and 'date' in entry and 'total_value' in entry:
            entry_date = entry['date']
            # Convert entry date to date object for consistent comparison
            if hasattr(entry_date, 'date'):
                entry_date = entry_date.date()
            
            if entry_date > cutoff_date and entry_date <= reference_date:
                total_value = entry.get('total_value', 0)
                if total_value is not None:
                    revenue_in_period += total_value
    
    return revenue_in_period

# Apply average unit price calculation 
sku_data['avg_unit_price'] = sku_data['historical_sales'].apply(get_avg_unit_price)


# Apply min/max unit price calculations
price_stats = sku_data['historical_sales'].apply(get_min_max_unit_price)
sku_data['min_unit_price'] = price_stats.apply(lambda x: x['min_unit_price'])
sku_data['max_unit_price'] = price_stats.apply(lambda x: x['max_unit_price'])

# Apply total revenue calculation
sku_data['total_revenue'] = sku_data['historical_sales'].apply(get_total_revenue)

# Apply revenue calculations for different time periods
sku_data['revenue_last_30d'] = sku_data['historical_sales'].apply(
    lambda x: get_revenue_last_n_days(x, now, 30)
)
sku_data['revenue_last_60d'] = sku_data['historical_sales'].apply(
    lambda x: get_revenue_last_n_days(x, now, 60)
)
sku_data['revenue_last_90d'] = sku_data['historical_sales'].apply(
    lambda x: get_revenue_last_n_days(x, now, 90)
)
sku_data['revenue_last_180d'] = sku_data['historical_sales'].apply(
    lambda x: get_revenue_last_n_days(x, now, 180)
)
sku_data['revenue_last_360d'] = sku_data['historical_sales'].apply(
    lambda x: get_revenue_last_n_days(x, now, 360)
)

# Calculate average daily revenue for each period
sku_data['avg_daily_revenue_30d'] = sku_data['revenue_last_30d'] / 30
sku_data['avg_daily_revenue_60d'] = sku_data['revenue_last_60d'] / 60
sku_data['avg_daily_revenue_90d'] = sku_data['revenue_last_90d'] / 90
sku_data['avg_daily_revenue_180d'] = sku_data['revenue_last_180d'] / 180
sku_data['avg_daily_revenue_360d'] = sku_data['revenue_last_360d'] / 360

# Display the new columns
sku_data

In [ ]:

# Basic velocity calculation
sku_data['velocity'] = sku_data['total_sales'] / sku_data['days_on_market'].replace(0, np.nan)

# Calculate velocity for different time periods (units per day)
sku_data['velocity_last_30d'] = sku_data['sales_last_30d'] / 30
sku_data['velocity_last_90d'] = sku_data['sales_last_90d'] / 90
sku_data['velocity_last_180d'] = sku_data['sales_last_180d'] / 180
sku_data['velocity_last_360d'] = sku_data['sales_last_360d'] / 360

# Function to calculate rolling 30-day velocity statistics with improved error handling
def calculate_velocity_stats(historical_sales):
    """Calculate max/min 30-day velocity and daily sales stats"""
    if isinstance(historical_sales, (list, np.ndarray)) and len(historical_sales) == 0:
        return {
            'max_30d_velocity_value': np.nan,
            'max_30d_velocity_range_start': np.nan,
            'max_30d_velocity_range_end': np.nan,
            'min_30d_velocity_value': np.nan,
            'min_30d_velocity_range_start': np.nan,
            'min_30d_velocity_range_end': np.nan,
            'max_daily_sales_value': 0,
            'max_daily_sales_date': np.nan,
            'min_daily_sales_value': 0,
            'min_daily_sales_date': np.nan
        }
    
    try:
        if len(historical_sales) == 0:
            return {
                'max_30d_velocity_value': np.nan,
                'max_30d_velocity_range_start': np.nan,
                'max_30d_velocity_range_end': np.nan,
                'min_30d_velocity_value': np.nan,
                'min_30d_velocity_range_start': np.nan,
                'min_30d_velocity_range_end': np.nan,
                'max_daily_sales_value': 0,
                'max_daily_sales_date': np.nan,
                'min_daily_sales_value': 0,
                'min_daily_sales_date': np.nan
            }
    except:
        return {
            'max_30d_velocity_value': np.nan,
            'max_30d_velocity_range_start': np.nan,
            'max_30d_velocity_range_end': np.nan,
            'min_30d_velocity_value': np.nan,
            'min_30d_velocity_range_start': np.nan,
            'min_30d_velocity_range_end': np.nan,
            'max_daily_sales_value': 0,
            'max_daily_sales_date': np.nan,
            'min_daily_sales_value': 0,
            'min_daily_sales_date': np.nan
        }
    
    try:
        # Filter entries that have both 'date' and 'quantity' keys
        valid_entries = []
        for entry in historical_sales:
            if isinstance(entry, dict) and 'date' in entry and 'quantity' in entry:
                valid_entries.append(entry)
        
        if len(valid_entries) == 0:
            return {
                'max_30d_velocity_value': np.nan,
                'max_30d_velocity_range_start': np.nan,
                'max_30d_velocity_range_end': np.nan,
                'min_30d_velocity_value': np.nan,
                'min_30d_velocity_range_start': np.nan,
                'min_30d_velocity_range_end': np.nan,
                'max_daily_sales_value': 0,
                'max_daily_sales_date': np.nan,
                'min_daily_sales_value': 0,
                'min_daily_sales_date': np.nan
            }
        
        # Convert to DataFrame and process
        sales_df = pd.DataFrame(valid_entries)
        sales_df['date'] = pd.to_datetime(sales_df['date'])
        
        # Handle timezone conversion
        if sales_df['date'].dt.tz is not None:
            sales_df['date'] = sales_df['date'].dt.tz_convert(None)
        
        sales_df = sales_df.sort_values('date')
        
        # Create date range and aggregate sales by date
        start_date = sales_df['date'].min()
        end_date = sales_df['date'].max()
        
        # If date range is less than 30 days, return NaN for velocity stats
        if (end_date - start_date).days < 30:
            # Still calculate daily stats
            sales_by_date = sales_df.groupby('date')['quantity'].sum()
            max_daily = sales_by_date.max()
            max_daily_date = sales_by_date.idxmax()
            min_daily = sales_by_date.min()
            min_daily_date = sales_by_date.idxmin()
            
            return {
                'max_30d_velocity_value': np.nan,
                'max_30d_velocity_range_start': np.nan,
                'max_30d_velocity_range_end': np.nan,
                'min_30d_velocity_value': np.nan,
                'min_30d_velocity_range_start': np.nan,
                'min_30d_velocity_range_end': np.nan,
                'max_daily_sales_value': max_daily,
                'max_daily_sales_date': max_daily_date,
                'min_daily_sales_value': min_daily,
                'min_daily_sales_date': min_daily_date
            }
        
        date_range = pd.date_range(start=start_date, end=end_date)
        sales_by_date = sales_df.groupby('date')['quantity'].sum().reindex(date_range, fill_value=0)
        
        # Calculate rolling 30-day velocity
        rolling_30d_sales = sales_by_date.rolling(window=30).sum()
        rolling_30d_velocity = rolling_30d_sales / 30
        
        # Find max and min 30-day velocity
        max_30d_velocity_value = rolling_30d_velocity.max()
        max_30d_velocity_date = rolling_30d_velocity.idxmax()
        min_30d_velocity_value = rolling_30d_velocity.min()
        min_30d_velocity_date = rolling_30d_velocity.idxmin()
        
        # Calculate date ranges
        max_30d_velocity_range_start = max_30d_velocity_date - pd.Timedelta(days=29)
        max_30d_velocity_range_end = max_30d_velocity_date
        min_30d_velocity_range_start = min_30d_velocity_date - pd.Timedelta(days=29)
        min_30d_velocity_range_end = min_30d_velocity_date
        
        # Find max and min daily sales
        max_daily_sales_value = sales_by_date.max()
        max_daily_sales_date = sales_by_date.idxmax()
        min_daily_sales_value = sales_by_date.min()
        min_daily_sales_date = sales_by_date.idxmin()
        
        return {
            'max_30d_velocity_value': max_30d_velocity_value,
            'max_30d_velocity_range_start': max_30d_velocity_range_start,
            'max_30d_velocity_range_end': max_30d_velocity_range_end,
            'min_30d_velocity_value': min_30d_velocity_value,
            'min_30d_velocity_range_start': min_30d_velocity_range_start,
            'min_30d_velocity_range_end': min_30d_velocity_range_end,
            'max_daily_sales_value': max_daily_sales_value,
            'max_daily_sales_date': max_daily_sales_date,
            'min_daily_sales_value': min_daily_sales_value,
            'min_daily_sales_date': min_daily_sales_date
        }
        
    except Exception as e:
        print(f"Error processing historical_sales: {e}")
        return {
            'max_30d_velocity_value': np.nan,
            'max_30d_velocity_range_start': np.nan,
            'max_30d_velocity_range_end': np.nan,
            'min_30d_velocity_value': np.nan,
            'min_30d_velocity_range_start': np.nan,
            'min_30d_velocity_range_end': np.nan,
            'max_daily_sales_value': np.nan,
            'max_daily_sales_date': np.nan,
            'min_daily_sales_value': np.nan,
            'min_daily_sales_date': np.nan
        }

# Apply velocity statistics calculation
velocity_stats = sku_data['historical_sales'].apply(calculate_velocity_stats)

# Extract individual columns from the velocity stats
sku_data['max_30d_velocity_value'] = velocity_stats.apply(lambda x: x['max_30d_velocity_value'])
sku_data['max_30d_velocity_range_start'] = velocity_stats.apply(lambda x: x['max_30d_velocity_range_start'])
sku_data['max_30d_velocity_range_end'] = velocity_stats.apply(lambda x: x['max_30d_velocity_range_end'])
sku_data['min_30d_velocity_value'] = velocity_stats.apply(lambda x: x['min_30d_velocity_value'])
sku_data['min_30d_velocity_range_start'] = velocity_stats.apply(lambda x: x['min_30d_velocity_range_start'])
sku_data['min_30d_velocity_range_end'] = velocity_stats.apply(lambda x: x['min_30d_velocity_range_end'])
sku_data['max_daily_sales_value'] = velocity_stats.apply(lambda x: x['max_daily_sales_value'])
sku_data['max_daily_sales_date'] = velocity_stats.apply(lambda x: x['max_daily_sales_date'])
sku_data['min_daily_sales_value'] = velocity_stats.apply(lambda x: x['min_daily_sales_value'])
sku_data['min_daily_sales_date'] = velocity_stats.apply(lambda x: x['min_daily_sales_date'])

sku_data


In [ ]:
inspect_dataset(sku_data)

In [ ]:
# INVENTORY AGE AND TURNOVER METRICS

def days_in_inventory(current_inventory, velocity_last_30d):
    """
    Calculate Days Sales in Inventory using recent 30-day velocity
    
    This metric tells you how many days it would take to sell all current inventory
    at the current sales pace (based on last 30 days of sales activity).
    
    Formula: DSI = Current Inventory / Daily Sales Velocity (last 30 days)
    
    Parameters:
    - current_inventory: Units currently in stock
    - velocity_last_30d: Average daily sales rate over last 30 days (units/day)
    
    Returns:
    - Number of days to sell current inventory at recent velocity
    - NaN if no recent sales velocity or no inventory
    """
    # Handle edge cases
    if pd.isna(current_inventory) or current_inventory == 0:
        return 0  # No inventory means 0 days to sell
    
    if pd.isna(velocity_last_30d) or velocity_last_30d == 0:
        return np.nan  # Can't calculate if no recent sales
    
    # DSI = Current Inventory / Recent Daily Sales Rate
    return current_inventory / velocity_last_30d


def calculate_inventory_turnover(total_sales, current_inventory, initial_inventory):
    """
    Calculate inventory turnover rate
    
    This measures how efficiently inventory is being sold by comparing total sales
    to average inventory levels. Higher values indicate better inventory efficiency.
    
    Formula: Inventory Turnover = Total Sales / Average Inventory
    where Average Inventory = (Current Inventory + Initial Inventory) / 2
    
    Parameters:
    - total_sales: Total units sold over product lifetime
    - current_inventory: Current units in stock
    - initial_inventory: Starting inventory when product was first stocked
    
    Returns:
    - Turnover ratio (times inventory was "turned over")
    - Higher values = more efficient inventory management
    """
    # Calculate average inventory level
    if pd.isna(initial_inventory) or initial_inventory == 0:
        # If no initial inventory data, use current inventory as proxy
        average_inventory = current_inventory
    else:
        # Use midpoint between initial and current inventory
        average_inventory = (current_inventory + initial_inventory) / 2
    
    # Handle edge cases
    if pd.isna(average_inventory) or average_inventory == 0:
        return np.nan  # Can't calculate turnover with no inventory
    
    if pd.isna(total_sales) or total_sales == 0:
        return 0  # No sales means zero turnover
    
    # Inventory Turnover = Total Sales / Average Inventory
    return total_sales / average_inventory



# APPLY METRICS TO DATAFRAME

print("Calculating inventory age and turnover metrics...")

# Calculate Days Sales in Inventory using recent 30-day velocity
# This tells us how long current inventory will last at current sales pace
sku_data['days_sales_in_inventory'] = sku_data.apply(
    lambda row: days_in_inventory(
        row['current_inventory'], 
        row['velocity_last_30d']
    ), 
    axis=1
)

# Calculate Inventory Turnover Rate
# This measures how efficiently we're converting inventory to sales
sku_data['inventory_turnover'] = sku_data.apply(
    lambda row: calculate_inventory_turnover(
        row['total_sales'], 
        row['current_inventory'], 
        row['initial_inventory']
    ), 
    axis=1
)

print("✓ Inventory metrics calculated successfully!")


# SUMMARY STATISTICS AND INSIGHTS

print("\n" + "="*60)
print("INVENTORY METRICS SUMMARY")
print("="*60)

# Days Sales in Inventory Statistics
print("\nDays Sales in Inventory (DSI) - Based on Last 30 Days Velocity:")
print(f"  Mean DSI: {sku_data['days_sales_in_inventory'].mean():.1f} days")
print(f"  Median DSI: {sku_data['days_sales_in_inventory'].median():.1f} days")
print(f"  75th percentile: {sku_data['days_sales_in_inventory'].quantile(0.75):.1f} days")
print(f"  90th percentile: {sku_data['days_sales_in_inventory'].quantile(0.90):.1f} days")

# Inventory Turnover Statistics  
print(f"\nInventory Turnover Rate:")
print(f"  Mean turnover: {sku_data['inventory_turnover'].mean():.2f}x")
print(f"  Median turnover: {sku_data['inventory_turnover'].median():.2f}x")
print(f"  25th percentile: {sku_data['inventory_turnover'].quantile(0.25):.2f}x")

# Identify slow-moving inventory based on DSI
slow_moving_threshold = 100  # SKUs that will take >90 days to sell at current pace
slow_moving_skus = sku_data[
    (sku_data['days_sales_in_inventory'] > slow_moving_threshold) & 
    (sku_data['current_inventory'] > 0)
]

print(f"\nSlow-Moving Inventory Alert:")
print(f"  SKUs taking >{slow_moving_threshold} days to sell: {len(slow_moving_skus)}")
print(f"  Total units in slow-moving inventory: {slow_moving_skus['current_inventory'].sum()}")

# Show top 10 slowest moving SKUs
if len(slow_moving_skus) > 0:
    print(f"\nTop 10 Slowest Moving SKUs:")
    slowest = slow_moving_skus.nlargest(10, 'days_sales_in_inventory')
    for idx, row in slowest.iterrows():
        print(f"  {row['sku']}: {row['days_sales_in_inventory']:.0f} days to sell {row['current_inventory']} units")

# Display the updated dataframe with new metrics
print(f"\nDataframe now includes new columns:")
print("  - days_sales_in_inventory: Days to sell current inventory at recent velocity")
print("  - inventory_turnover: How many times inventory turns over (higher = better)")

# Show sample of the new columns
print(f"\nSample of new metrics:")
sample_cols = ['sku', 'name', 'current_inventory', 'velocity_last_30d', 
               'days_sales_in_inventory', 'inventory_turnover']
print(sku_data[sample_cols].head(10))


In [ ]:
# Check if unit_price is always equal to max_unit_price
price_comparison = sku_data[['unit_price', 'max_unit_price']].copy()
price_comparison['prices_equal'] = price_comparison['unit_price'] == price_comparison['max_unit_price']

print("Unit Price vs Max Unit Price Analysis:")
print(f"Total rows: {len(price_comparison)}")
print(f"Rows where unit_price == max_unit_price: {price_comparison['prices_equal'].sum()}")
print(f"Rows where unit_price != max_unit_price: {(~price_comparison['prices_equal']).sum()}")
print(f"Rows with NaN unit_price: {price_comparison['unit_price'].isna().sum()}")
print(f"Rows with NaN max_unit_price: {price_comparison['max_unit_price'].isna().sum()}")

# Show examples where they differ
different_prices = price_comparison[~price_comparison['prices_equal'] & price_comparison['unit_price'].notna() & price_comparison['max_unit_price'].notna()]
if len(different_prices) > 0:
    print(f"\nExamples where prices differ:")
    print(different_prices.head(10))

# Show all rows where max_unit_price is greater than unit_price
max_greater_than_unit = sku_data[
    (sku_data['max_unit_price'] > sku_data['unit_price']) & 
    sku_data['unit_price'].notna() & 
    sku_data['max_unit_price'].notna()
]


# Fill nan unit_price values with max_unit_price
sku_data['unit_price'] = sku_data['unit_price'].fillna(sku_data['max_unit_price'])
max_greater_than_unit

In [ ]:
inspect_dataset(sku_data)
sku_data

In [ ]:
df = sku_data
df_sku = sku_data[['sku', 'current_inventory', 'cost_per_unit', 'avg_unit_price', 'velocity']]

# Add total_cost column
df_sku['total_cost'] = df_sku['current_inventory'] * df_sku['cost_per_unit']

# Rename columns
df_sku = df_sku.rename(columns={
    'sku': 'SKU',
    'current_inventory': 'on_hand',
    'cost_per_unit': 'cost_per_unit',
    'total_cost': 'total_cost',
    'avg_unit_price': 'avg_price',
    'velocity': 'sales_velocity'
})

df_sku

# Set variables
# Define discount progression once at the beginning
STARTING_DISCOUNT_PCT = 40  # Starting discount percentage
WEEKLY_DISCOUNT_INCREMENT = 5  # Discount increase per week

# Vitality LTM Data 
Payroll = 1405788.35  
plExpense = 0
Occupancy = 3007483.62
Miscellaneous = 517152.08  

# Caluclate for 12 weeks 
Payroll = Payroll/52 
plExpense = plExpense/52 
Occupancy = Occupancy/52 
Miscellaneous = Miscellaneous/52
Advertising = 10000
Liquidation = 3500
print(Payroll)
print(plExpense)
print(Occupancy)
print(Miscellaneous)
print(Advertising)
print(Liquidation)
TotalExpenses = Payroll + plExpense + Occupancy + Miscellaneous + Advertising + Liquidation
print(TotalExpenses)




In [ ]:
# =============================================================================
# DYNAMIC LIQUIDATION ANALYSIS WITH VELOCITY MULTIPLIER MODEL
# =============================================================================


def calculate_sku_velocities_at_discounts(df_sku, max_weeks=20):
    """
    Calculate velocity for each SKU at specific discount levels by week.a
    Dynamic weeks starting at 30% discount, adding 5% every week until max_weeks
    
    Args:
        df_sku: DataFrame with SKU data including sales_velocity
        max_weeks: Maximum number of weeks to calculate (default 20)
    
    Returns:
        DataFrame with original columns plus new velocity columns at different discount levels
    """
    
    # Define discount levels dynamically using the global constants
    discount_levels = [STARTING_DISCOUNT_PCT + (i * WEEKLY_DISCOUNT_INCREMENT) for i in range(max_weeks)]
    
    # Start with a copy of the original dataframe
    result_df = df_sku.copy()
    
    # For each discount level, calculate the velocity
    for discount in discount_levels:
        velocities = []
        for _, row in df_sku.iterrows():
            base_velocity = row['sales_velocity']
            
            # Calculate velocity multiplier using the Hills equation
            # y = 1 + (U-1) * x^n / (C^n + x^n)
            # Parameters: U, n, and C
            x = discount
            U = 5.0  # Maximum velocity multiplier
            n = 5.0   # Hill coefficient (cooperativity)
            C = 50.0  # Half-maximal discount percentage
            
            velocity_multiplier = 1 + (U - 1) * (x ** n) / (C ** n + x ** n)
                
            predicted_velocity = base_velocity * velocity_multiplier
            
            velocities.append(round(predicted_velocity, 3))
        
        result_df[f'velocity_at_{discount}%'] = velocities
    
    return result_df



def simulate_dynamic_liquidation(df_enhanced, total_expenses, max_weeks=20):
    """
    Simulate dynamic liquidation period with increasing discounts.
    Stops when weekly gross recovery minus dynamic fee drops below total_expenses.
    Starting at defined discount percentage, adding increment every week.
    
    Args:
        df_enhanced: DataFrame with SKU data and velocity calculations
        total_expenses: Weekly expense threshold to stop liquidation (excluding fee)
        max_weeks: Maximum weeks to simulate (safety limit)
    
    Returns:
        Tuple: (liquidation_df, actual_weeks_simulated, summary_data, final_week_stopped)
    """
    
    # Create base DataFrame with SKU and initial inventory
    liquidation_df = df_enhanced[['SKU', 'on_hand']].copy()
    liquidation_df = liquidation_df.rename(columns={'on_hand': 'Week_0_Inventory'})
    
    # Track remaining inventory for each week
    remaining_inventory = df_enhanced['on_hand'].copy()
    
    # Initialize summary data
    summary_data = []
    
    # Get SKU data for calculations
    sku_data = df_enhanced[['SKU', 'on_hand', 'cost_per_unit', 'avg_price']].copy()
    
    week = 1
    continue_liquidation = True
    final_week_stopped = None
    
    print(f"🚀 Starting dynamic liquidation simulation...")
    print(f"💰 Weekly expense threshold (excluding fee): ${total_expenses:,.2f}")
    print(f"⏹️  Liquidation will stop when (gross recovery - 5% fee) < ${total_expenses:,.2f}")
    print()
    
    while continue_liquidation and week <= max_weeks:
        discount_pct = STARTING_DISCOUNT_PCT + ((week - 1) * WEEKLY_DISCOUNT_INCREMENT)
        
        # Check if we have velocity data for this discount level
        velocity_col = f'velocity_at_{discount_pct}%'
        if velocity_col not in df_enhanced.columns:
            print(f"⚠️  No velocity data for week {week} (discount {discount_pct}%). Stopping simulation.")
            break
        
        # Calculate weekly sales
        weekly_sales = df_enhanced[velocity_col] * 7  # 7 days per week
        
        # Calculate units sold this week (before updating inventory)
        prev_inventory = remaining_inventory.copy()
        remaining_inventory = (remaining_inventory - weekly_sales).clip(lower=0)
        units_sold_by_sku = prev_inventory - remaining_inventory
        
        # Calculate financial metrics for this week
        inventory_cost = (units_sold_by_sku * sku_data['cost_per_unit']).sum()
        inventory_sp = (units_sold_by_sku * sku_data['avg_price']).sum()
        
        # Calculate gross recovery (applying discount)
        selling_price_discounted = sku_data['avg_price'] * (1 - discount_pct / 100)
        gross_recovery = (units_sold_by_sku * selling_price_discounted).sum()
        
        # Calculate dynamic fee (5% of gross recovery)
        dynamic_fee = 0.05 * gross_recovery
        net_recovery = gross_recovery - dynamic_fee
        
        # Store inventory results
        liquidation_df[f'Week_{week}_Inventory'] = remaining_inventory.round(0).astype(int)
        
        # Store summary data
        pct_of_cost = (gross_recovery / inventory_cost * 100) if inventory_cost > 0 else 0
        pct_of_sp = (gross_recovery / inventory_sp * 100) if inventory_sp > 0 else 0
        
        # Check if we should continue (net recovery >= total expenses)
        if net_recovery < total_expenses:
            print(f"🛑 Stopping liquidation at week {week}:")
            print(f"   Gross recovery: ${gross_recovery:,.2f}")
            print(f"   Dynamic fee (5%): ${dynamic_fee:,.2f}")
            print(f"   Net recovery: ${net_recovery:,.2f}")
            print(f"   Total expenses: ${total_expenses:,.2f}")
            print(f"   Discount level: {discount_pct:.1f}%")
            final_week_stopped = week
            continue_liquidation = False
        else:
            # Only add profitable weeks to summary data
            summary_data.append({
                'Week': week,
                'Units_Sold': units_sold_by_sku.sum(),
                'Inventory_Cost': inventory_cost,
                'Inventory_SP': inventory_sp,
                'Discount_Pct': discount_pct,
                'Gross_Recovery': gross_recovery,
                'Dynamic_Fee': dynamic_fee,
                'Net_Recovery': net_recovery,
                'Pct_of_Cost': pct_of_cost,
                'Pct_of_SP': pct_of_sp
            })
            print(f"✅ Week {week}: Net recovery ${net_recovery:,.2f} > ${total_expenses:,.2f} expenses ({discount_pct:.1f}% discount)")
        
        week += 1
    
    actual_weeks = len(summary_data)  # Only count profitable weeks
    print(f"\n📊 Liquidation simulation completed after {actual_weeks} profitable weeks")
    if final_week_stopped:
        print(f"📈 Week {final_week_stopped} would have been unprofitable and was excluded")
    
    return liquidation_df, actual_weeks, summary_data, final_week_stopped


# Calculate velocities for all SKUs at extended discount levels
print("🎯 Calculating dynamic sigmoid velocity model for all SKUs...")
print(f"📈 Model: velocity = base_velocity × sigmoid_multiplier(discount)")
print(f"   Sigmoid equation: y = 22.605 * x^5.153 / (78.069^5.153 + x^5.153) + 0.981")
print(f"   Where x = discount percentage and y = velocity multiplier")
print(f"📊 Discount progression: starts at {STARTING_DISCOUNT_PCT:.1f}%, increases by {WEEKLY_DISCOUNT_INCREMENT:.1f}% per week")
print()

# Check if df_sku is properly defined before proceeding
if 'df_sku' not in locals() or df_sku is None:
    print("❌ ERROR: df_sku is not defined!")
    print("🔧 SOLUTION: Run the previous cell first to properly define df_sku")
    raise NameError("df_sku is not defined. Please run the previous cell first.")

# Calculate velocities for up to 20 weeks (safety buffer)
df_sku_enhanced = calculate_sku_velocities_at_discounts(df_sku, max_weeks=20)

print(f"✅ Calculated velocities for {len(df_sku_enhanced)} SKUs for up to 20 weeks")
print()

# Run the dynamic liquidation simulation
df_liquidation, actual_weeks, summary_data, final_week_stopped = simulate_dynamic_liquidation(
    df_sku_enhanced, 
    TotalExpenses,  # Use the calculated weekly expense threshold (excluding fee)
    max_weeks=20
)

print(f"\n🔍 Sample results (first 5 SKUs) for {actual_weeks} weeks:")
# Show sample columns dynamically based on actual weeks simulated
sample_cols = ['SKU', 'Week_0_Inventory', 'Week_1_Inventory']
if actual_weeks >= 2:
    sample_cols.append('Week_2_Inventory')
if actual_weeks >= 5:
    sample_cols.append('Week_5_Inventory')
if actual_weeks >= 8:
    sample_cols.append('Week_8_Inventory')
sample_cols.append(f'Week_{actual_weeks}_Inventory')

print(df_liquidation[sample_cols].head())


# Calculate total starting inventory cost for percentage calculations
sku_data = df_sku_enhanced[['SKU', 'on_hand', 'cost_per_unit', 'avg_price']].copy()
total_starting_cost = (df_liquidation['Week_0_Inventory'] * sku_data['cost_per_unit']).sum()

# Create and display the formatted table (only profitable weeks)
print(f"\n📊 DYNAMIC LIQUIDATION ANALYSIS TABLE (Profitable Weeks Only)")
print("=" * 90)
print(f"{'Week':<6} {'Inventory':<15} {'% of':<8} {'Inventory at':<15} {'Gross':<15} {'% of':<8} {'% of':<8}")
print(f"{'':6} {'Cost $':<15} {'Liquidated':<8} {'SP $':<15} {'Recovery $':<15} {'Cost':<8} {'SP':<8}")
print("-" * 90)

# Calculate totals (only from profitable weeks)
total_inv_cost = sum(data['Inventory_Cost'] for data in summary_data)
total_inv_sp = sum(data['Inventory_SP'] for data in summary_data)
total_gross_recovery = sum(data['Gross_Recovery'] for data in summary_data)
total_dynamic_fee = sum(data['Dynamic_Fee'] for data in summary_data)
total_net_recovery = sum(data['Net_Recovery'] for data in summary_data)
total_pct_cost = (total_gross_recovery / total_inv_cost * 100) if total_inv_cost > 0 else 0
total_pct_sp = (total_gross_recovery / total_inv_sp * 100) if total_inv_sp > 0 else 0
total_pct_of_liquidated = (total_inv_cost / total_inv_cost * 100) if total_inv_cost > 0 else 0  # This will be 100%

for data in summary_data:
    week = data['Week']
    inv_cost = data['Inventory_Cost']
    pct_liquidated = (data['Inventory_Cost'] / total_inv_cost * 100) if total_inv_cost > 0 else 0  # % of liquidated inventory, not total inventory
    inv_sp = data['Inventory_SP']
    gross_rec = data['Gross_Recovery']
    pct_cost = data['Pct_of_Cost']
    pct_sp = data['Pct_of_SP']
    
    print(f"{week:<6} ${inv_cost:<14,.0f} {pct_liquidated:<7.1f}% ${inv_sp:<14,.0f} ${gross_rec:<14,.0f} {pct_cost:<7.1f}% {pct_sp:<7.1f}%")



print("-" * 90)
print(f"{'TOTAL':<6} ${total_inv_cost:<14,.0f} {total_pct_of_liquidated:<7.1f}% ${total_inv_sp:<14,.0f} ${total_gross_recovery:<14,.0f} {total_pct_cost:<7.1f}% {total_pct_sp:<7.1f}%")

print()
print("💡 Key Insights:")
print(f"   • Starting inventory value: ${total_starting_cost:,.0f} at cost")
print(f"   • Inventory liquidated in profitable weeks: ${total_inv_cost:,.0f} ({(total_inv_cost/total_starting_cost*100) if total_starting_cost>0 else 0:.1f}% of total)")
print(f"   • Expected total gross recovery: ${total_gross_recovery:,.0f} ({total_pct_cost:.1f}% of liquidated cost)")
print(f"   • Total dynamic fees (5%): ${total_dynamic_fee:,.0f}")
print(f"   • Expected total net recovery: ${total_net_recovery:,.0f} ({(total_net_recovery/total_inv_cost*100) if total_inv_cost>0 else 0:.1f}% of liquidated cost)")
print(f"   • Profitable liquidation weeks: {actual_weeks}")
if final_week_stopped:
    print(f"   • Week {final_week_stopped} would have been unprofitable and was excluded")
discount_progression = [f"{STARTING_DISCOUNT_PCT + ((i-1) * WEEKLY_DISCOUNT_INCREMENT):.0f}%" for i in range(1, actual_weeks + 1)]
print(f"   • Discount progression: {', '.join(discount_progression)}")
print(f"   • Weekly expense threshold (excluding fee): ${TotalExpenses:,.2f}")

# Show inventory aging analysis
print()
print("📈 DYNAMIC INVENTORY AGING BREAKDOWN:")
print("-" * 50)

# Calculate what percentage of inventory sells out in each time period
total_starting_units = df_liquidation['Week_0_Inventory'].sum()

print(f"{'Age Bucket':<15} {'Units':<10} {'% of Total':<12} {'Cumulative %':<15}")
print("-" * 50)

cumulative_sold = 0

# Week 1
if actual_weeks >= 1:
    units_sold = df_liquidation['Week_0_Inventory'].sum() - df_liquidation['Week_1_Inventory'].sum()
    cumulative_sold += units_sold
    pct_total = (units_sold / total_starting_units * 100) if total_starting_units > 0 else 0
    cumulative_pct = (cumulative_sold / total_starting_units * 100) if total_starting_units > 0 else 0
    print(f"{'Week 1':<15} {units_sold:<10,} {pct_total:<11.1f}% {cumulative_pct:<14.1f}%")

# Week 2
if actual_weeks >= 2:
    units_sold = df_liquidation['Week_1_Inventory'].sum() - df_liquidation['Week_2_Inventory'].sum()
    cumulative_sold += units_sold
    pct_total = (units_sold / total_starting_units * 100) if total_starting_units > 0 else 0
    cumulative_pct = (cumulative_sold / total_starting_units * 100) if total_starting_units > 0 else 0
    print(f"{'Week 2':<15} {units_sold:<10,} {pct_total:<11.1f}% {cumulative_pct:<14.1f}%")

# Weeks 3 to middle point
if actual_weeks >= 3:
    mid_week = min(6, actual_weeks)
    if mid_week > 2:
        start_inventory = df_liquidation['Week_2_Inventory'].sum()
        end_inventory = df_liquidation[f'Week_{mid_week}_Inventory'].sum()
        units_sold = start_inventory - end_inventory
        cumulative_sold += units_sold
        pct_total = (units_sold / total_starting_units * 100) if total_starting_units > 0 else 0
        cumulative_pct = (cumulative_sold / total_starting_units * 100) if total_starting_units > 0 else 0
        print(f"{'Weeks 3-' + str(mid_week):<15} {units_sold:<10,} {pct_total:<11.1f}% {cumulative_pct:<14.1f}%")

# Remaining weeks
if actual_weeks > 6:
    start_inventory = df_liquidation['Week_6_Inventory'].sum()
    end_inventory = df_liquidation[f'Week_{actual_weeks}_Inventory'].sum()
    units_sold = start_inventory - end_inventory
    cumulative_sold += units_sold
    pct_total = (units_sold / total_starting_units * 100) if total_starting_units > 0 else 0
    cumulative_pct = (cumulative_sold / total_starting_units * 100) if total_starting_units > 0 else 0
    print(f"{f'Weeks 7-{actual_weeks}':<15} {units_sold:<10,} {pct_total:<11.1f}% {cumulative_pct:<14.1f}%")

# Remaining inventory
remaining = df_liquidation[f'Week_{actual_weeks}_Inventory'].sum()
remaining_pct = (remaining / total_starting_units * 100) if total_starting_units > 0 else 0
print(f"{'Remaining':<15} {remaining:<10,} {remaining_pct:<11.1f}% {'100.0%':<14}")

print()
print(f"🎯 DYNAMIC LIQUIDATION SUMMARY:")
print(f"   • Profitable liquidation weeks: {actual_weeks}")
if final_week_stopped:
    print(f"   • Simulation stopped at week {final_week_stopped} (would have been unprofitable)")
print(f"   • Total units liquidated in profitable weeks: {total_starting_units - remaining:,} ({100 - remaining_pct:.1f}%)")
print(f"   • SKUs completely sold out: {(df_liquidation[f'Week_{actual_weeks}_Inventory'] == 0).sum()} out of {len(df_liquidation)}")
print(f"   • Weekly expense threshold (excluding fee): ${TotalExpenses:,.2f}")
if summary_data:
    print(f"   • Final profitable week gross recovery: ${summary_data[-1]['Gross_Recovery']:,.2f}")
    print(f"   • Final profitable week dynamic fee (5%): ${summary_data[-1]['Dynamic_Fee']:,.2f}")
    print(f"   • Final profitable week net recovery: ${summary_data[-1]['Net_Recovery']:,.2f}")
print(f"   • Liquidation efficiency: Net recovery covered expenses for {actual_weeks} weeks")


In [ ]:
# Create Plotly visualizations for the liquidation analysis

# Use the summary_data from the main liquidation analysis table above
# summary_data is a list of dicts, one per week, with keys:
# 'Week', 'Units_Sold', 'Inventory_Cost', 'Inventory_SP', 'Discount_Pct', 'Gross_Recovery', 'Pct_of_Cost', 'Pct_of_SP'

# Extract data for plotting
weeks = [d['Week'] for d in summary_data]
inv_costs = [d['Inventory_Cost'] for d in summary_data]
gross_recoveries = [d['Gross_Recovery'] for d in summary_data]
pct_of_costs = [d['Pct_of_Cost'] for d in summary_data]
pct_of_sps = [d['Pct_of_SP'] for d in summary_data]
inv_sps = [d['Inventory_SP'] for d in summary_data]

# 1. Graph: % of Total Inventory Cost Liquidated Each Week (as in the table: Inventory_Cost / total_starting_cost)
total_starting_cost = (df_liquidation['Week_0_Inventory'] * 
                       df_liquidation.merge(df_sku_enhanced[['SKU', 'cost_per_unit']], on='SKU')['cost_per_unit']).sum()
pct_of_total_cost = [(cost / total_starting_cost * 100) if total_starting_cost > 0 else 0 for cost in inv_costs]

fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=weeks,
    y=pct_of_total_cost,
    mode='lines+markers',
    name='% of Total Inventory Cost Liquidated',
    line=dict(color='rgb(55, 83, 109)', width=3),
    marker=dict(size=8, color='rgb(55, 83, 109)'),
    text=[f'{pct:.1f}%' for pct in pct_of_total_cost],
    textposition='top center'
))
fig1.update_layout(
    title=f'Dynamic Liquidation Analysis - % of Total Inventory Cost Liquidated ({actual_weeks} Weeks)',
    xaxis_title='Week',
    yaxis_title='% of Total Inventory Cost',
    showlegend=False,
    height=500,
    width=900,
    template='plotly_white'
)
fig1.update_xaxes(tickmode='linear', tick0=1, dtick=1)
fig1.show()

# 2. Graph: Recovery Rate as % of Original Selling Price (Pct_of_SP from summary_data)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=weeks,
    y=pct_of_sps,
    mode='lines+markers',
    name='% of SP Recovery',
    line=dict(color='rgb(255, 65, 54)', width=3),
    marker=dict(size=8, color='rgb(255, 65, 54)'),
    text=[f'{pct:.1f}%' for pct in pct_of_sps],
    textposition='top center'
))
fig2.update_layout(
    title=f'Dynamic Recovery Analysis - % of Original Selling Price ({actual_weeks} Weeks)',
    xaxis_title='Week',
    yaxis_title='% of Original Selling Price',
    showlegend=False,
    height=500,
    width=900,
    template='plotly_white'
)
fig2.update_xaxes(tickmode='linear', tick0=1, dtick=1)
fig2.show()

# 3. Graph: Recovery Rate as % of Original Cost (Pct_of_Cost from summary_data)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=weeks,
    y=pct_of_costs,
    mode='lines+markers',
    name='% of Cost Recovery',
    line=dict(color='rgb(50, 171, 96)', width=3),
    marker=dict(size=8, color='rgb(50, 171, 96)'),
    text=[f'{pct:.1f}%' for pct in pct_of_costs],
    textposition='top center'
))
fig3.update_layout(
    title=f'Dynamic Recovery Analysis - % of Original Cost ({actual_weeks} Weeks)',
    xaxis_title='Week',
    yaxis_title='% of Original Cost',
    showlegend=False,
    height=500,
    width=900,
    template='plotly_white'
)
fig3.update_xaxes(tickmode='linear', tick0=1, dtick=1)
fig3.show()

print("📊 Dynamic Liquidation Analysis Visualizations Complete!")
print("   • Graph 1: Weekly percentage of total inventory cost liquidated")
print("   • Graph 2: Recovery rate as percentage of original selling price")
print("   • Graph 3: Recovery rate as percentage of original cost")
print(f"   • Analysis covers {actual_weeks} weeks until recovery dropped below ${TotalExpenses:,.2f} expense threshold")

In [ ]:
#Run this to sample rows for testing
#sku_data = sku_data.sample(n=50, random_state=None)
#sku_data

In [ ]:
# Display first 10 rows of sku_data
#print("First 10 rows of sku_data:")
#print(sku_data.head(10).to_string())


In [ ]:
sku_data = df

sku_data
inspect_dataset(sku_data)

In [ ]:
inspect_dataset(sku_data)

In [ ]:
MODEL = "gpt-4.1"  

# 1) DISCOVERY  ─ returns {"attr": [examples,…], …}
# ─────────────────────────────────────────────────────────
DISCOVER_SCHEMA = {
    "name": "list_attributes",
    "parameters": {
        "type": "object",
        "required": ["attributes"],
        "properties": {
            "attributes": {
                "type": "object",
                "additionalProperties": {
                    "type": "array",
                    "items": {"type": ["string", "number", "boolean", "null"]}
                }
            }
        }
    }
}


DISCOVER_SYSTEM = """
You are a generic attribute‐discovery AI.  
Given a list of product names, return a JSON mapping:
  "attributes": { "<attr_name>": [<example_values…>] }  
— use snake_case keys for booleans (is_ or has_) and pascal_case for everything else.  
— values may be strings, numbers, or booleans. booleans are preferred when possible. For boolean attributes, have them start with is_ or has_ and be true or false. 
— don’t invent attributes, only those present or logically implied. Don't be over specific, be general.
- do not repeat the same attributes
- do not incude none as an attribute
- for string attributes, dont make the attributes overlap with each other, so an attribute  shouldnt have multiplte strings apply to it. 
- for outlier products, add product_type as a string attribute for those (be sure to incude the main product itself in this list), and feel free to use null for those for the other string and number (very rarly boolean)attributes that are not applicable.
- attributes shouldnt generally be more than 2 words
- attributes should be mutually exclusive
Return only the function call:
```python
list_attributes(attributes={...})
"""

def discover_attributes(names: pd.Series, retries=2) -> dict:
    bullets = "\n".join(f"- {n}" for n in names.tolist())
    user_prompt = f"Sample SKU names:\n{bullets}\n"

    for _ in range(retries):
        r = openai.chat.completions.create(
            model=MODEL,
            temperature=0,
            tools=[{"type": "function", "function": DISCOVER_SCHEMA}],
            tool_choice={"type": "function",
                         "function": {"name": "list_attributes"}},
            messages=[
                {"role": "system", "content": DISCOVER_SYSTEM},
                {"role": "user",   "content": user_prompt},
            ],
        )

        payload = json.loads(
            r.choices[0].message.tool_calls[0].function.arguments
        )
        attrs = payload["attributes"]
        if attrs:                      # got a non-empty dict
            return attrs

    # if we fall through the loop, raise instead of returning None
    raise RuntimeError("Attribute discovery failed after retries")

DISCOVERED = discover_attributes(sku_data["name"])   # save this
# right after discovery, strip duplicates but keep order:
for k, vals in DISCOVERED.items():
    DISCOVERED[k] = list(dict.fromkeys(vals))

from collections import defaultdict

def format_discovered_attributes(discovered: dict):
    type_groups = defaultdict(list)

    for attr, values in discovered.items():
        # Check the type of the first non-null value
        sample_type = type(next((v for v in values if v is not None), None))

        if sample_type is bool:
            type_groups['Booleans'].append((attr, values))
        elif sample_type in [int, float]:
            type_groups['Numbers'].append((attr, values))
        else:
            type_groups['Strings'].append((attr, values))

    for category in ['Booleans', 'Numbers', 'Strings']:
        print(f"\n## {category}\n")
        for attr, values in sorted(type_groups[category]):
            print(f"{attr}: {values}")

# Call the formatting function
format_discovered_attributes(DISCOVERED)


In [ ]:
# ─────────────────────────────────────────────────────────
# 2)  EXTRACTION  (patched)
# ─────────────────────────────────────────────────────────
def make_extract_schema(attr_keys: list[str], discovered: dict) -> dict:
    props = {
        k: {"enum": list(discovered.get(k, [])) + [None]}
        for k in attr_keys
    }

    return {
        "name": "extract_attributes",
        "description": "Return attributes for a batch of SKUs in order.",
        "parameters": {
            "type": "object",
            "properties": {
                "items": {                     # top-level list of SKU objects
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": props,
                        "required": attr_keys,
                        "additionalProperties": False,
                    },
                }
            },
            "required": ["items"],
        },
    }


EXTRACT_SYSTEM = """
You are an attribute-extraction AI.

Return exactly one call to `extract_attributes` whose arguments are:

{
  "items": [
     {"color": "Black", "is_readers": true, ...},
     ...
  ]
}

Strings / numbers / booleans only; unknown → null.
For boolean attributes (is_/has_): true if implied, false if absent.
For string and number attributes, null is very rare.
Boolean attributes should be true if implied, false if absent, never null.
There might be some error/mistakes in the data, so use your best judgement.
"""

def extract_batch(names: list[str], attr_keys: list[str], schema) -> list[dict]:
    bullets = "\n".join(f"- {n}" for n in names)
    user_msg = (
        "For each SKU below, return an *array* of objects with these keys:\n"
        f"{', '.join(attr_keys)}\n"
        "One object per SKU, same order:\n"
        + bullets
    )

    r = openai.chat.completions.create(
        model=MODEL,
        temperature=0,
        tools=[{"type": "function", "function": schema}],     # ← wrap!
        tool_choice={"type": "function",
                     "function": {"name": "extract_attributes"}},
        messages=[
            {"role": "system", "content": EXTRACT_SYSTEM},
            {"role": "user",   "content": user_msg},
        ],
    )

    # Tool call payload → JSON
    args_json = r.choices[0].message.tool_calls[0].function.arguments
    raw_list = json.loads(args_json)["items"]

    # pad / reorder
    return [{k: d.get(k, None) for k in attr_keys} for d in raw_list]

# ─────────────────────────────────────────────────────────
# 3) PIPELINE  ─ enrich df + one‑hot selected attrs
# ─────────────────────────────────────────────────────────
def enrich_df(
    df: pd.DataFrame,
    name_col: str = "name",
    force_attrs: tuple[str, ...] = (),
    one_hot: tuple[str, ...] = (),
) -> pd.DataFrame:
    """
    • discovers attributes,
    • extracts them for every SKU,
    • one‑hots selected attributes,
    • returns df with new columns.
    """
    discovered = DISCOVERED
    print("Discovered:", discovered.keys())
    # merge discovered + forced list (keeps order, no dups)
    attr_keys = list(dict.fromkeys(list(discovered.keys()) + list(force_attrs)))
    schema = make_extract_schema(attr_keys, discovered)

        # ----- run GPT in 10-SKU batches (≈10× faster) -----
    BATCH = 10
    rows = []
  
    for i in tqdm(range(0, len(df), BATCH), desc="Extracting batches"):
        chunk = df[name_col].iloc[i : i + BATCH].tolist()
        batch_results = extract_batch(chunk, attr_keys, schema)
        rows.extend(batch_results)
        
    
    feat = pd.DataFrame(rows, columns=attr_keys)  # ensures all cols exist
    
    feat = feat.where(pd.notnull(feat), pd.NA)
    feat = (pd.DataFrame(rows, columns=attr_keys)
          .astype(object)              # keeps Python None
          .where(pd.notnull, None))    # convert any stray NaN → None


    # ----- one‑hot -----
    for col in one_hot:
        if col not in feat:          # skip if discovery missed it
            continue
        dummies = pd.get_dummies(feat[col]).astype(bool)
        dummies.columns = [f"{col}__{c}" for c in dummies.columns]
        feat = pd.concat([feat.drop(columns=[col]), dummies], axis=1)
    
    # After one-hot encoding
    feat = feat.loc[:, feat.any()]

    # ----- return enriched -----
    return pd.concat([df.reset_index(drop=True), feat], axis=1)


# ─────────────────────────────────────────────────────────
# 4) RUN
# ─────────────────────────────────────────────────────────
sku_enriched = enrich_df(sku_data)

# peek
print(sku_enriched.columns[len(sku_data.columns):])  # Print only the new columns
print(sku_enriched.columns[len(sku_data.columns):].to_list())  # Print the names of the new columns
sku_enriched[["name"] + list(sku_enriched.columns[len(sku_data.columns):])]  # Print the first few rows of the new columns along with the name column

In [ ]:
print("Extracting new columns (excluding original sku_data columns):")
new_columns = sku_enriched.columns[len(sku_data.columns):].to_list()
print("New columns:", new_columns)

print("\nPreview of 'name' column along with the new columns:")
print(sku_enriched[["name"] + new_columns].head())

# Calculate and print the number of null values for each column
print("\nNull counts per column:")
subset = sku_enriched[["name"] + new_columns]
null_counts = subset.isnull().sum()
for col, count in null_counts.items():
    print(f"• {col}: {count} null(s)")

# For each column, print the names of the SKUs where null values occur
print("\nSKUs with null values for each column:")
for col in subset.columns:
    null_skus = subset[subset[col].isnull()]["name"].tolist()
    print(f"• {col}: {null_skus}")

# Special check for the 'name' column (it should not have any nulls)
if sku_enriched["name"].isnull().any():
    print("\nWARNING: 'name' column contains null values! Please check the above SKUs.")

In [ ]:
sku_enriched

In [ ]:
# identify attribute columns 
# “base” = every column that already existed in the raw sku_data table
base_cols   = set(sku_data.columns)
attr_cols   = [c for c in sku_enriched.columns if c not in base_cols]

# classify by type
def _is_bool(col):
    s = sku_enriched[col].dropna().unique()
    return (len(s) <= 2) and set(s).issubset({True, False})

bool_cols = [c for c in attr_cols if _is_bool(c)]

# treat anything with a *manageable* number of distinct values as a category.
cat_cols  = [c for c in attr_cols
             if c not in bool_cols]

# everything else is ignored for bar‑plots but can still be used later if needed
print("AI generated columns:", attr_cols)
print("Boolean flags       :", bool_cols)
print("Categorical attrs   :", cat_cols)
print("Base collumns:", base_cols)

In [ ]:
def make_summary(df, group_col):
    summary = (df.groupby(group_col, dropna=False)
               .agg(sku_count=("sku", "nunique"),
                    total_sales=("total_sales", "sum")))
    # Add a helper column to flag rows where the group key is null
    summary["_null"] = summary.index.to_series().apply(lambda x: pd.isna(x))
    # Sort by: non-null groups first (i.e. _null=False) and then by ttotal_sale descending
    summary = summary.sort_values(by=["_null", "total_sales"], ascending=[True, False])
    return summary.drop(columns=["_null"])

def create_interactive_chart(df, group_col, title):
    """Create an interactive Plotly chart for the summary data"""
    summary = make_summary(df, group_col)
    
    # Convert index to string and handle NaN values
    labels = [str(x) if pd.notna(x) else 'Unknown/Missing' for x in summary.index]
    
    # Create subplot with secondary y-axis
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('SKU Count', 'Total Sales'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Add bar charts
    fig.add_trace(
        go.Bar(
            x=labels,
            y=summary['sku_count'],
            name='SKU Count',
            marker_color='lightblue',
            text=summary['sku_count'],
            textposition='auto',
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=labels,
            y=summary['total_sales'],
            name='Total Sales',
            marker_color='lightcoral',
            text=summary['total_sales'],
            textposition='auto',
        ),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        title_text=f"{title} - Distribution Analysis",
        title_x=0.5,
        showlegend=False,
        height=400,
        margin=dict(l=50, r=50, t=80, b=50)
    )
    
    # Update x-axis labels
    fig.update_xaxes(title_text="Category", row=1, col=1)
    fig.update_xaxes(title_text="Category", row=1, col=2)
    fig.update_yaxes(title_text="Number of SKUs", row=1, col=1)
    fig.update_yaxes(title_text="Total Sales", row=1, col=2)
    
    return fig, summary

# Process boolean columns with interactive charts
for col in bool_cols:
    print(f"\n── {col.upper()} ─────────────────────────────────────")
    fig, summary = create_interactive_chart(sku_enriched, col, col.upper())
    fig.show()
    print("\nSummary Table:")
    display(summary)

# Process categorical columns with interactive charts
for col in cat_cols:
    print(f"\n── {col.upper()} ─────────────────────────────────────")
    fig, summary = create_interactive_chart(sku_enriched, col, col.upper())
    fig.show()
    print("\nSummary Table:")
    display(summary)

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as pcolors
from colorsys import hsv_to_rgb
# ----------------- HELPER FUNCTIONS (REFACTORED FOR PLOTLY) ---------------------
def make_numeric_palette(levels, cmap_name="Viridis"):
    """
    Map numeric categories onto a continuous Plotly colour map.
    """
    nums = pd.to_numeric(levels, errors="coerce")
    if nums.isna().all():
        return ["#CCCCCC"] * len(levels)
    
    # Get a Plotly continuous colorscale
    cmap = getattr(pcolors.sequential, cmap_name)
    
    # Normalize the numeric values to map to the colorscale
    min_val, max_val = nums.min(), nums.max()
    if min_val == max_val: # Avoid division by zero if all values are the same
        norm_vals = [0.5] * len(nums)
    else:
        norm = (nums - min_val) / (max_val - min_val)
        norm_vals = norm.fillna(0.5) # Default to middle color for NaNs
    
    # Map normalized values to colors
    return [pcolors.sample_colorscale(cmap, val)[0] if pd.notna(val) else "#CCCCCC" for val in norm_vals]

def default_palette(n):
    """
    Use a distinct Plotly color palette for better readability.
    """
    if n <= 10:
        return pcolors.qualitative.T10[:n]
    else:
        # Use a longer palette and cycle it if needed
        extended = pcolors.qualitative.Vivid + pcolors.qualitative.Light24
        return (extended * (n // len(extended) + 1))[:n]

import plotly.graph_objects as go # Make sure this import is at the top of your script

def get_color(name: str) -> str:
    """
    Checks if a color name is recognized by Plotly.
    - If yes, returns the color name.
    - If no, generates a unique, deterministic color from the name.
    """
    try:
        # Let Plotly's own validator try to recognize the color
        go.Figure().add_scatter(marker_color=[name])
        return name
    except ValueError:
        # If Plotly raises an error, the color is unknown.
        # Generate a color by hashing the name (your original fallback method).
        hue = (abs(hash(name)) % 360) / 360.0
        rgb_float = hsv_to_rgb(hue, 0.8, 0.9) # Sightly adjusted for good contrast
        rgb_int = tuple(int(c * 255) for c in rgb_float)
        return f'#%02x%02x%02x' % rgb_int

BOOL_COLOR_MAP = {
    True:  "#2CA02C",  # green
    False: "#D62728",  # red
    "True":  "#2CA02C",
    "False": "#D62728",
    "Unknown": "#CCCCCC",
    "nan": "#CCCCCC",
    pd.NA: "#CCCCCC",
    None: "#CCCCCC",
}

def choose_bar_colours(grp_name: str, levels: pd.Series) -> list[str]:
    """
    Decide colouring strategy based on attribute type using Plotly palettes.
    """
    if grp_name in bool_cols:
        return levels.map(BOOL_COLOR_MAP).fillna("#CCCCCC").tolist()

    if pd.to_numeric(levels, errors="coerce").notna().all():
        return make_numeric_palette(levels)

    if "color" in grp_name.lower():
        return [get_color(v) if pd.notna(v) else "#CCCCCC" for v in levels]

    return default_palette(len(levels))

# Enhanced metrics with better descriptions
metrics = {
    "Current Stock": "current_inventory",
    "Initial Stock": "initial_inventory",
    "Sales (30d)": "sales_last_30d",
    "Total Sales": "total_sales",
    "Total Revenue": "total_revenue",
    "Revenue (30d)": "revenue_last_30d",
    "Daily Revenue (30d)": "avg_daily_revenue_30d"
}

# ----------------- DATA PROCESSING (UNCHANGED) -----------------------------
def calculate_attribute_variability(df, attr_col, metrics_dict):
    """Calculate variability by summing absolute differences from mean for each option within an attribute using percentages"""
    df_clean = df.copy()
    df_clean[attr_col] = df_clean[attr_col].fillna("Other")
    if df_clean[attr_col].dtype == 'bool' or df_clean[attr_col].dtype == 'boolean' or any(isinstance(v, (bool, np.bool_)) for v in df_clean[attr_col].dropna().unique()):
        df_clean[attr_col] = df_clean[attr_col].astype(str)
    grouped = df_clean.groupby(attr_col)[list(metrics_dict.values())].sum()
    if grouped.empty or grouped.sum().sum() == 0: return 0
    pct_grouped = grouped.div(grouped.sum(axis=0), axis=1).fillna(0) * 100
    total_attribute_variability = 0
    for option in pct_grouped.index:
        option_values = pct_grouped.loc[option].values
        option_mean = option_values.mean()
        option_variability = sum(abs(value - option_mean) for value in option_values)
        total_attribute_variability += option_variability
    return total_attribute_variability

attributes = [attr for attr in attr_cols if attr in sku_enriched.columns and 2 <= sku_enriched[attr].fillna("Other").nunique()]
print(f"Selected attributes for portfolio analysis: {attributes}")

print("Calculating attribute variability...")
attribute_variabilities = {attr: calculate_attribute_variability(sku_enriched, attr, metrics) for attr in attributes}
sorted_attributes = sorted(attribute_variabilities.items(), key=lambda x: x[1], reverse=True)
attributes = [attr for attr, score in sorted_attributes]
print(f"\nAttributes ranked by variability (highest first):")
for i, (attr, score) in enumerate(sorted_attributes, 1):
    print(f"  {i}. {attr.replace('_', ' ').title()}: {score:.1f}")

if not attributes:
    print("No suitable attributes found for portfolio analysis.")
else:
    attribute_titles = {attr: attr.replace("_", " ").title() for attr in attributes}

    def format_number(num):
        if num >= 1_000_000: return f'{num/1_000_000:.1f}M'
        if num >= 1_000: return f'{num/1_000:.1f}K'
        return f'{int(num)}'

    def get_top5_categories_by_sales(attr, reference_metric="total_sales"):
        df_temp = sku_enriched.copy()
        df_temp[attr] = df_temp[attr].fillna("Other")
        if df_temp[attr].dtype == 'bool' or df_temp[attr].dtype == 'boolean':
            df_temp[attr] = df_temp[attr].astype(str)
        top_by_sales = df_temp.groupby(attr)[reference_metric].sum().sort_values(ascending=False)
        return top_by_sales.head(5).index.tolist()

    attribute_color_maps = {}
    for attr in attributes:
        top5_categories = get_top5_categories_by_sales(attr)
        all_categories = top5_categories + ["Others"]
        colors = choose_bar_colours(attr, pd.Series(all_categories))
        attribute_color_maps[attr] = {category: colors[i] if i < len(colors) else '#CCCCCC' for i, category in enumerate(all_categories)}

    # ----------------- PLOTLY VISUALIZATION --------------------------------------
    fig = make_subplots(
        rows=len(attributes),
        cols=len(metrics),
        subplot_titles=list(metrics.keys()),
        shared_xaxes=True,
        vertical_spacing=0.01,
        horizontal_spacing=0.01
    )

    for r, attr in enumerate(attributes):
        top5_categories = get_top5_categories_by_sales(attr)
        color_map = attribute_color_maps[attr]
        attr_variability = attribute_variabilities[attr]
        print(f"\nProcessing: {attribute_titles[attr]} - Variability Score: {attr_variability:.1f}")

        for c, (metric_name, metric_col) in enumerate(metrics.items()):
            # Aggregate data
            df_temp = sku_enriched.copy()
            df_temp[attr] = df_temp[attr].fillna("Other")
            if df_temp[attr].dtype == 'bool' or df_temp[attr].dtype == 'boolean':
                df_temp[attr] = df_temp[attr].astype(str)
            
            df_full = df_temp.groupby(attr)[metric_col].sum().reset_index().rename(columns={attr: "level"})
            df_top5 = df_full[df_full["level"].isin(top5_categories)].copy()
            df_others = df_full[~df_full["level"].isin(top5_categories)]

            if not df_others.empty and df_others[metric_col].sum() > 0:
                 df_top5 = pd.concat([df_top5, pd.DataFrame([{"level": "Others", metric_col: df_others[metric_col].sum()}])], ignore_index=True)

            sort_map = {cat: i for i, cat in enumerate(top5_categories + ['Others'])}
            df_top5['sort_order'] = df_top5['level'].map(sort_map)
            df_top5 = df_top5.sort_values('sort_order').reset_index(drop=True)

            total_value = df_top5[metric_col].sum()
            df_top5['percentage'] = (df_top5[metric_col] / total_value * 100) if total_value > 0 else 0

            # Add bar traces for the stacked bar
            for _, row in df_top5.iterrows():
                fig.add_trace(go.Bar(
                    x=[0], # Single bar at x=0
                    y=[row['percentage']],
                    name=row['level'],
                    marker_color=color_map.get(row['level'], '#CCCCCC'),
                    text=f"{row['percentage']:.1f}%",
                    textposition='inside',
                    insidetextanchor='middle',
                    hoverinfo='text',
                    hovertext=f"{row['level']}: {format_number(row[metric_col])} ({row['percentage']:.1f}%)",
                    textfont={'color': 'white' if row['percentage'] > 5 else 'rgba(0,0,0,0)', 'size': 10} # Hide small labels
                ), row=r+1, col=c+1)

            # Add total value annotation at the top
            if total_value > 0:
                fig.add_annotation(
                    x=0, y=103, text=f"<b>{format_number(total_value)}</b>",
                    showarrow=False, bgcolor="lightgray", opacity=0.8,
                    font=dict(size=11), row=r+1, col=c+1
                )

        # Set Y-axis title for the first column of each row
        fig.update_yaxes(title_text=f"<b>{attribute_titles[attr]}</b>", row=r+1, col=1)


    # ----------------- FIGURE LAYOUT AND STYLING ---------------------------------
    fig.update_layout(
        title={
            'text': "<b>Product Portfolio Analysis: Sales & Inventory Breakdown</b><br><sub>Attributes sorted by variability (highest first)</sub>",
            'y': 0.98,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 20}
        },
        barmode='stack',
        showlegend=False,
        height=350 * len(attributes), # Dynamic height
        width=1200,
        margin=dict(l=120, r=20, t=80, b=20),
        plot_bgcolor='white'
    )

    # Style axes for all subplots
    fig.update_xaxes(
        showticklabels=False,
        showgrid=False,
        zeroline=False
    )
    fig.update_yaxes(
        range=[0, 110],
        showgrid=True,
        gridcolor='#e5e5e5',
        tickvals=[0, 25, 50, 75, 100],
        ticktext=['0%', '25%', '50%', '75%', '100%'],
        zeroline=True,
        zerolinecolor='#333',
        zerolinewidth=1.5
    )
    # Hide all Y-axis ticks and labels except for the first column
    for c in range(2, len(metrics) + 1):
        fig.update_yaxes(showticklabels=False, col=c)


    fig.show()

    # ----------------- SUMMARY STATISTICS (UNCHANGED) ---------------------------
    print("\n" + "="*100)
    print("PRODUCT PORTFOLIO ANALYSIS SUMMARY (SORTED BY VARIABILITY)")
    print("="*100)
    print(f"\nOVERALL METRICS:")
    for metric_name, metric_col in metrics.items():
        total = sku_enriched[metric_col].sum()
        print(f"  • {metric_name}: {format_number(total)}")
    print(f"\nATTRIBUTE VARIABILITY RANKING:")
    for i, (attr, var_score) in enumerate(sorted_attributes, 1):
        print(f"  {i}. {attr.replace('_', ' ').title()}: {var_score:.1f}")


In [ ]:
inspect_dataset(sku_enriched)

In [ ]:
print("🔄 Creating sales_long DataFrame from historical_sales...")

# Function to extract daily sales records from historical_sales
def extract_sales_records(sku_enriched):
    """Extract individual sales records from historical_sales into long format"""
    records = []
    
    for idx, row in sku_enriched.iterrows():
        sku = row['sku']
        historical_sales = row['historical_sales']
        
        # Skip if no historical sales data
        if not isinstance(historical_sales, (list, np.ndarray)) or len(historical_sales) == 0:
            continue
            
        # Extract each sales record
        for sale_record in historical_sales:
            if isinstance(sale_record, dict) and 'date' in sale_record and 'quantity' in sale_record:
                # Safely get unit_price and quantity, ensuring no None values
                quantity = sale_record.get('quantity', 0)
                unit_price = sale_record.get('unit_price', 0)
                
                # Convert None values to 0
                if quantity is None:
                    quantity = 0
                if unit_price is None:
                    unit_price = 0
                
                # Parse date more carefully
                try:
                    if isinstance(sale_record['date'], str):
                        # Try different date formats
                        for date_format in ['%Y-%m-%d', '%Y/%m/%d', '%m/%d/%Y', '%d/%m/%Y']:
                            try:
                                parsed_date = pd.to_datetime(sale_record['date'], format=date_format)
                                break
                            except ValueError:
                                continue
                        else:
                            # If no format works, use pandas parser
                            parsed_date = pd.to_datetime(sale_record['date'])
                    else:
                        parsed_date = pd.to_datetime(sale_record['date'])
                except (ValueError, TypeError):
                    print(f"Warning: Could not parse date {sale_record['date']} for SKU {sku}")
                    continue
                
                # Create record with all attributes from the SKU row
                record = {
                    'sku': sku,
                    'date': parsed_date,
                    'quantity': quantity,
                    'unit_price': unit_price,
                    'revenue': quantity * unit_price
                }
                
                # Add all other columns from sku_enriched row to preserve attributes
                for col in sku_enriched.columns:
                    if col not in ['sku', 'historical_sales']:
                        record[col] = row[col]
                
                records.append(record)
    
    return records

# Extract records and create sales_long DataFrame
records = extract_sales_records(sku_enriched)
sales_long = pd.DataFrame(records)

sales_long

In [ ]:
inspect_dataset(sales_long)

In [ ]:
# 🎛️ INTERACTIVE SALES DASHBOARD - Comprehensive Analysis Interface
import plotly.graph_objects as go
import plotly.colors
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import pandas as pd

# Check if sales_long has data
if not sales_long.empty:
    # Ensure date column is properly formatted
    sales_long['date'] = pd.to_datetime(sales_long['date'])
    sales_long = sales_long.sort_values('date').reset_index(drop=True)
    
    print(f"✅ Created sales_long DataFrame with {len(sales_long)} sales records from {sales_long['sku'].nunique()} SKUs")
    print(f"📋 Available columns in sales_long: {list(sales_long.columns)}")
    
    # Data quality checks
    print(f"\n📊 Data Quality Checks:")
    print(f"  • Date range: {sales_long['date'].min().strftime('%Y-%m-%d')} to {sales_long['date'].max().strftime('%Y-%m-%d')}")
    print(f"  • Total sales records: {len(sales_long):,}")
    print(f"  • Unique SKUs: {sales_long['sku'].nunique()}")
    print(f"  • Total quantity sold: {sales_long['quantity'].sum():,.0f}")
    print(f"  • Total revenue: ${sales_long['revenue'].sum():,.2f}")
    print(f"  • Records with zero quantity: {(sales_long['quantity'] == 0).sum()}")
    print(f"  • Records with missing dates: {sales_long['date'].isna().sum()}")
    
    # Show sample of recent data
    print(f"\n📝 Sample of most recent sales:")
    recent_sample = sales_long.nlargest(5, 'date')[['sku', 'date', 'quantity', 'unit_price', 'revenue']]
    for _, row in recent_sample.iterrows():
        print(f"  • {row['sku']}: {row['date'].strftime('%Y-%m-%d')} - {row['quantity']} units @ ${row['unit_price']:.2f} = ${row['revenue']:.2f}")
    
    print(f"\n🏷️ Available attribute columns for breakdown: {attr_cols}")
    
    # Create Interactive Dashboard Class
    class SalesDashboard:
        def __init__(self, sales_data, attribute_columns):
            self.sales_data = sales_data
            self.attr_cols = attribute_columns
            self.output_widget = widgets.Output()
            
            # Create control widgets
            self.create_controls()
            self.create_dashboard_layout()
            
        def create_controls(self):
            """Create all control widgets"""
            # Attribute selection dropdown - only Overall and attr_cols
            dropdown_options = [('Overall (No Breakdown)', 'none')]
            for col in self.attr_cols:
                dropdown_options.append((col.replace('_', ' ').title(), col))
            
            self.attr_dropdown = widgets.Dropdown(
                options=dropdown_options,
                value='none',
                description='Breakdown by:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='300px')
            )
            
            # Metric selection dropdown
            self.metric_dropdown = widgets.Dropdown(
                options=[
                    ('Total Units Sold', 'quantity'),
                    ('Total Revenue', 'revenue'),
                    ('Percentage of Units', 'quantity_pct'),
                    ('Percentage of Revenue', 'revenue_pct')
                ],
                value='quantity',
                description='Metric:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='300px')
            )
            
            # Rolling average slider
            self.rolling_slider = widgets.IntSlider(
                value=1,
                min=1,
                max=30,
                step=1,
                description='Rolling Avg (days):',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='400px')
            )
            
            # Update button
            self.update_button = widgets.Button(
                description='🔄 Update Chart',
                button_style='primary',
                layout=widgets.Layout(width='150px')
            )
            
            # Bind events
            self.update_button.on_click(self.update_charts)
            
        def create_dashboard_layout(self):
            """Create the dashboard layout"""
            # Control panel
            controls_box = widgets.VBox([
                widgets.HTML("<h3>📊 Sales Dashboard Controls</h3>"),
                widgets.HBox([self.attr_dropdown, self.metric_dropdown]),
                self.rolling_slider,
                self.update_button,
                widgets.HTML("<hr>")
            ])
            
            # Main dashboard
            self.dashboard = widgets.VBox([
                controls_box,
                self.output_widget
            ])
            
        def prepare_data(self, attr_name, metric_name, rolling_days):
            """Prepare data based on selected parameters - only use attr_cols"""
            data = self.sales_data.copy()
            
            if attr_name == 'none':
                # Overall data - no breakdown
                if metric_name in ['quantity', 'revenue']:
                    daily_data = data.groupby('date')[metric_name].sum().reset_index()
                    if rolling_days > 1:
                        daily_data[f'{metric_name}_smooth'] = daily_data[metric_name].rolling(window=rolling_days, center=True).mean()
                        return daily_data, None, f"Total {metric_name.title()}"
                    return daily_data, None, f"Daily {metric_name.title()}"
                else:
                    # For percentage, show 100% line
                    daily_data = data.groupby('date')['quantity'].sum().reset_index()
                    daily_data['pct'] = 100.0
                    return daily_data, None, "100% (No Breakdown)"
            else:
                # Breakdown by attribute - ONLY if it's in attr_cols
                if attr_name not in self.attr_cols:
                    raise ValueError(f"Invalid breakdown attribute: {attr_name}. Must be one of {self.attr_cols}")
                
                # Check if the attribute column exists in the data
                if attr_name not in data.columns:
                    raise ValueError(f"Column '{attr_name}' not found in sales data")
                
                if metric_name in ['quantity_pct', 'revenue_pct']:
                    # Percentage breakdown
                    base_metric = metric_name.replace('_pct', '')
                    daily_totals = data.groupby('date')[base_metric].sum()
                    daily_breakdown = data.groupby(['date', attr_name])[base_metric].sum().unstack(fill_value=0)
                    
                    # Calculate percentages
                    pct_breakdown = daily_breakdown.div(daily_totals, axis=0) * 100
                    
                    if rolling_days > 1:
                        pct_breakdown = pct_breakdown.rolling(window=rolling_days, center=True).mean()
                    
                    return pct_breakdown, daily_breakdown, f"Percentage of {base_metric.title()}"
                else:
                    # Absolute values breakdown
                    daily_breakdown = data.groupby(['date', attr_name])[metric_name].sum().unstack(fill_value=0)
                    
                    if rolling_days > 1:
                        daily_breakdown = daily_breakdown.rolling(window=rolling_days, center=True).mean()
                    
                    # Also calculate totals
                    daily_totals = daily_breakdown.sum(axis=1)
                    
                    return daily_breakdown, daily_totals, f"Total {metric_name.title()}"
                    
        def create_chart(self, data, totals, attr_name, metric_name, rolling_days):
            """Create the appropriate chart based on parameters"""
            if attr_name == 'none':
                # Single line chart
                fig = go.Figure()
                
                metric_col = f'{metric_name}_smooth' if rolling_days > 1 and f'{metric_name}_smooth' in data.columns else metric_name
                
                fig.add_trace(go.Scatter(
                    x=data['date'],
                    y=data[metric_col] if metric_col in data.columns else data['pct'],
                    mode='lines',
                    name=f'{metric_name.title()}',
                    line=dict(color='#2E86AB', width=2),
                    fill='tonexty'
                ))
                
                title = f"📈 {metric_name.replace('_', ' ').title()}"
                if rolling_days > 1:
                    title += f" ({rolling_days}-day rolling average)"
                    
            elif metric_name.endswith('_pct'):
                # Stacked area chart for percentages
                fig = go.Figure()
                
                colors = plotly.colors.qualitative.Set3[:len(data.columns)]
                
                for i, category in enumerate(data.columns):
                    fig.add_trace(go.Scatter(
                        x=data.index,
                        y=data[category],
                        mode='lines',
                        name=str(category),
                        stackgroup='one',
                        line=dict(width=0),
                        fillcolor=colors[i % len(colors)]
                    ))
                
                fig.update_layout(
                    yaxis=dict(title='Percentage (%)', range=[0, 100]),
                    hovermode='x unified'
                )
                
                title = f"📊 Percentage Breakdown by {attr_name.replace('_', ' ').title()}"
                if rolling_days > 1:
                    title += f" ({rolling_days}-day rolling average)"
                    
            else:
                # Stacked area chart for absolute values
                fig = go.Figure()
                
                colors = plotly.colors.qualitative.Set3[:len(data.columns)]
                
                for i, category in enumerate(data.columns):
                    fig.add_trace(go.Scatter(
                        x=data.index,
                        y=data[category],
                        mode='lines',
                        name=str(category),
                        stackgroup='one',
                        line=dict(width=0),
                        fillcolor=colors[i % len(colors)]
                    ))
                
                # Add total line if available
                if totals is not None:
                    fig.add_trace(go.Scatter(
                        x=data.index,
                        y=totals,
                        mode='lines',
                        name='Total',
                        line=dict(color='black', width=2, dash='dash')
                    ))
                
                title = f"📈 {metric_name.title()} Breakdown by {attr_name.replace('_', ' ').title()}"
                if rolling_days > 1:
                    title += f" ({rolling_days}-day rolling average)"
            
            # Common layout settings
            fig.update_layout(
                title=title,
                xaxis_title='Date',
                width=1000,
                height=600,
                showlegend=True,
                legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
            )
            
            return fig
            
        def update_charts(self, button_click=None):
            """Update charts based on current selections"""
            with self.output_widget:
                clear_output(wait=True)
                
                try:
                    # Get current selections
                    attr_name = self.attr_dropdown.value
                    metric_name = self.metric_dropdown.value
                    rolling_days = self.rolling_slider.value
                    
                    print("🔄 Updating dashboard...")
                    print("=" * 80)
                    print(f"🎯 Selected breakdown: {attr_name if attr_name != 'none' else 'Overall'}")
                    print(f"📊 Available attr_cols: {self.attr_cols}")
                    
                    # Prepare data
                    main_data, totals, chart_title = self.prepare_data(attr_name, metric_name, rolling_days)
                    
                    # Create and display chart
                    fig = self.create_chart(main_data, totals, attr_name, metric_name, rolling_days)
                    fig.show()
                    
                    # Print summary information
                    print("=" * 80)
                    rolling_text = f" with {rolling_days}-day rolling average" if rolling_days > 1 else ""
                    breakdown_text = f"by {attr_name.replace('_', ' ').title()}" if attr_name != 'none' else "Overall"
                    print(f"🎯 Chart: {breakdown_text} - {metric_name.replace('_', ' ').title()}{rolling_text}")
                    
                    # Data summary - Fix date extraction logic
                    if attr_name == 'none':
                        # For overall data, use 'date' column
                        data_start = main_data['date'].min().strftime('%Y-%m-%d')
                        data_end = main_data['date'].max().strftime('%Y-%m-%d')
                        total_days = len(main_data)
                    else:
                        # For breakdown data, use index (which are dates)
                        if hasattr(main_data.index, 'min') and hasattr(main_data.index.min(), 'strftime'):
                            data_start = main_data.index.min().strftime('%Y-%m-%d')
                            data_end = main_data.index.max().strftime('%Y-%m-%d')
                        else:
                            # Fallback if index is not datetime
                            data_start = str(main_data.index.min())
                            data_end = str(main_data.index.max())
                        total_days = len(main_data.index)
                        
                    print(f"📅 Time Range: {data_start} to {data_end} ({total_days} days)")
                    print(f"📊 Breakdown: {breakdown_text}")
                    print(f"📈 Metric: {metric_name.replace('_', ' ').title()}")
                    
                    # Show category breakdown if applicable
                    if attr_name != 'none' and hasattr(main_data, 'columns'):
                        categories = main_data.columns.tolist()
                        print(f"🏷️  Categories ({len(categories)}): {', '.join(str(c) for c in categories)}")
                        print(f"🔍 Breakdown attribute: {attr_name} (from attr_cols)")
                    
                    print("=" * 80)
                    
                except Exception as e:
                    print(f"❌ Error creating charts: {e}")
                    import traceback
                    traceback.print_exc()
        
        def display(self):
            """Display the dashboard"""
            display(self.dashboard)
            # Auto-generate initial chart
            self.update_charts(None)

    # Create and display the dashboard
    if attr_cols:
        print("\n🚀 Creating Interactive Sales Dashboard...")
        print(f"📋 Dashboard breakdown options:")
        print(f"   • Overall (no breakdown)")
        for col in attr_cols:
            print(f"   • {col.replace('_', ' ').title()}")
        print(f"\n🔒 Only these {len(attr_cols)} attribute columns will be available for breakdown")
        dashboard = SalesDashboard(sales_long, attr_cols)
        dashboard.display()
    else:
        print("❌ No suitable attribute columns found for dashboard creation.")
        print("   Available columns:", list(sales_long.columns))
        print("   Base columns excluded:", base_cols)
        
else:
    print("❌ No data available for dashboard creation - sales_long DataFrame is empty.")

# 📊 Private Credit & Product Analysis Visualizations

This section provides comprehensive visualizations for private credit analysis and product performance evaluation, focusing on sales trends, revenue patterns, pricing dynamics, inventory metrics, and velocity analysis. Created by Claude


In [ ]:
# 1. Sales Performance Analysis - Time Series Overview
print("🏪 SALES PERFORMANCE ANALYSIS")
print("="*50)

# Create figure with subplots for sales metrics
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Sales Performance Analysis Across Time Periods', fontsize=16, fontweight='bold')

# Sales volume by time period
sales_periods = ['sales_last_30d', 'sales_last_90d', 'sales_last_180d', 'sales_last_360d', 'total_sales']
sales_values = [sku_enriched[col].sum() for col in sales_periods]
time_labels = ['30 Days', '90 Days', '180 Days', '360 Days', 'Total']

ax1.bar(time_labels, sales_values, color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#8B5A2B'])
ax1.set_title('Total Sales Volume by Time Period', fontweight='bold')
ax1.set_ylabel('Total Sales Units')
ax1.tick_params(axis='x', rotation=45)
for i, v in enumerate(sales_values):
    ax1.text(i, v + max(sales_values)*0.01, f'{v:,.0f}', ha='center', va='bottom', fontweight='bold')

# Distribution of sales performance (30-day focus)
ax2.hist(sku_enriched['sales_last_30d'], bins=20, color='#2E86AB', alpha=0.7, edgecolor='black')
ax2.set_title('Distribution of 30-Day Sales Performance', fontweight='bold')
ax2.set_xlabel('Sales Units (Last 30 Days)')
ax2.set_ylabel('Number of Products')
ax2.axvline(sku_enriched['sales_last_30d'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {sku_enriched["sales_last_30d"].mean():.1f}')
ax2.legend()

# Sales velocity comparison
velocities = ['velocity_last_30d', 'velocity_last_90d', 'velocity_last_180d', 'velocity_last_360d']
velocity_means = [sku_enriched[col].mean() for col in velocities]
velocity_labels = ['30d Velocity', '90d Velocity', '180d Velocity', '360d Velocity']

ax3.plot(velocity_labels, velocity_means, marker='o', linewidth=3, markersize=8, color='#C73E1D')
ax3.fill_between(velocity_labels, velocity_means, alpha=0.3, color='#C73E1D')
ax3.set_title('Average Sales Velocity Trends', fontweight='bold')
ax3.set_ylabel('Units per Day')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

# Top performers vs bottom performers
top_performers = sku_enriched.nlargest(10, 'sales_last_30d')[['name', 'sales_last_30d']].head(5)
bottom_performers = sku_enriched.nsmallest(10, 'sales_last_30d')[['name', 'sales_last_30d']].head(5)

y_pos = range(len(top_performers))
ax4.barh(y_pos, top_performers['sales_last_30d'], color='#2E86AB', alpha=0.8)
ax4.set_yticks(y_pos)
ax4.set_yticklabels([name[:20] + '...' if len(name) > 20 else name for name in top_performers['name']], fontsize=8)
ax4.set_title('Top 5 Products by 30-Day Sales', fontweight='bold')
ax4.set_xlabel('Sales Units (Last 30 Days)')

plt.tight_layout()
plt.show()

# Print key statistics
print(f"\n📈 KEY SALES METRICS:")
print(f"• Total sales across all products: {sku_enriched['total_sales'].sum():,} units")
print(f"• Average 30-day sales per product: {sku_enriched['sales_last_30d'].mean():.1f} units")
print(f"• Products with zero 30-day sales: {(sku_enriched['sales_last_30d'] == 0).sum()}")
print(f"• Best performing product (30d): {sku_enriched.loc[sku_enriched['sales_last_30d'].idxmax(), 'name']} ({sku_enriched['sales_last_30d'].max()} units)")
print(f"• Average sales velocity: {sku_enriched['velocity'].mean():.3f} units/day")


In [ ]:
# 2. Revenue Analysis - Financial Performance Dashboard
print("💰 REVENUE ANALYSIS")
print("="*50)

# Create comprehensive revenue dashboard
fig = plt.figure(figsize=(18, 14))
gs = gridspec.GridSpec(3, 3, figure=fig, hspace=0.3, wspace=0.3)

# Revenue trends across time periods
ax1 = fig.add_subplot(gs[0, :2])
revenue_periods = ['revenue_last_30d', 'revenue_last_60d', 'revenue_last_90d', 'revenue_last_180d', 'revenue_last_360d', 'total_revenue']
revenue_values = [sku_enriched[col].sum() for col in revenue_periods]
revenue_labels = ['30 Days', '60 Days', '90 Days', '180 Days', '360 Days', 'Total']

bars = ax1.bar(revenue_labels, revenue_values, color=['#1B4332', '#2D6A4F', '#40916C', '#52B788', '#74C69D', '#95D5B2'])
ax1.set_title('Total Revenue by Time Period', fontsize=14, fontweight='bold')
ax1.set_ylabel('Revenue ($)')
ax1.tick_params(axis='x', rotation=45)
# Add value labels on bars
for bar, value in zip(bars, revenue_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + max(revenue_values)*0.01, 
             f'${value:,.0f}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# Daily revenue trends
ax2 = fig.add_subplot(gs[0, 2])
daily_revenues = ['avg_daily_revenue_30d', 'avg_daily_revenue_60d', 'avg_daily_revenue_90d', 
                  'avg_daily_revenue_180d', 'avg_daily_revenue_360d']
daily_values = [sku_enriched[col].mean() for col in daily_revenues]
daily_labels = ['30d', '60d', '90d', '180d', '360d']

ax2.plot(daily_labels, daily_values, marker='o', linewidth=3, markersize=8, color='#2D6A4F')
ax2.fill_between(daily_labels, daily_values, alpha=0.3, color='#2D6A4F')
ax2.set_title('Avg Daily Revenue Trends', fontweight='bold')
ax2.set_ylabel('Daily Revenue ($)')
ax2.grid(True, alpha=0.3)

# Revenue distribution (last 30 days)
ax3 = fig.add_subplot(gs[1, 0])
ax3.hist(sku_enriched['revenue_last_30d'], bins=25, color='#40916C', alpha=0.7, edgecolor='black')
ax3.set_title('30-Day Revenue Distribution', fontweight='bold')
ax3.set_xlabel('Revenue ($)')
ax3.set_ylabel('Number of Products')
ax3.axvline(sku_enriched['revenue_last_30d'].mean(), color='red', linestyle='--', linewidth=2, 
           label=f'Mean: ${sku_enriched["revenue_last_30d"].mean():.0f}')
ax3.legend()

# Revenue concentration (Pareto analysis)
ax4 = fig.add_subplot(gs[1, 1])
sorted_revenue = sku_enriched['total_revenue'].sort_values(ascending=False)
cumulative_pct = (sorted_revenue.cumsum() / sorted_revenue.sum() * 100)
ax4.plot(range(1, len(cumulative_pct) + 1), cumulative_pct, color='#1B4332', linewidth=2)
ax4.axhline(y=80, color='red', linestyle='--', label='80% Revenue')
ax4.axvline(x=len(cumulative_pct) * 0.2, color='orange', linestyle='--', label='20% Products')
ax4.set_title('Revenue Concentration (Pareto)', fontweight='bold')
ax4.set_xlabel('Product Rank')
ax4.set_ylabel('Cumulative Revenue %')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Top revenue generators
ax5 = fig.add_subplot(gs[1, 2])
top_revenue = sku_enriched.nlargest(8, 'revenue_last_30d')[['name', 'revenue_last_30d']]
y_pos = range(len(top_revenue))
bars = ax5.barh(y_pos, top_revenue['revenue_last_30d'], color='#52B788')
ax5.set_yticks(y_pos)
ax5.set_yticklabels([name[:15] + '...' if len(name) > 15 else name for name in top_revenue['name']], fontsize=9)
ax5.set_title('Top Revenue Products (30d)', fontweight='bold')
ax5.set_xlabel('Revenue ($)')

# Revenue vs Sales correlation
ax6 = fig.add_subplot(gs[2, 0])
ax6.scatter(sku_enriched['sales_last_30d'], sku_enriched['revenue_last_30d'], 
           alpha=0.6, color='#74C69D', edgecolors='black', linewidth=0.5)
ax6.set_title('Revenue vs Sales Correlation', fontweight='bold')
ax6.set_xlabel('Sales Units (30d)')
ax6.set_ylabel('Revenue (30d)')
ax6.grid(True, alpha=0.3)

# Add correlation coefficient
correlation = sku_enriched['sales_last_30d'].corr(sku_enriched['revenue_last_30d'])
ax6.text(0.05, 0.95, f'Correlation: {correlation:.3f}', transform=ax6.transAxes, 
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8), fontweight='bold')

# Average unit price analysis
ax7 = fig.add_subplot(gs[2, 1])
# Use available price columns and create 30d average from recent sales data
current_avg_price = sku_enriched['unit_price'].mean()
# Calculate weighted average price from recent revenue/sales ratio where sales > 0
recent_avg_price = (sku_enriched[sku_enriched['sales_last_30d'] > 0]['revenue_last_30d'] / 
                   sku_enriched[sku_enriched['sales_last_30d'] > 0]['sales_last_30d']).mean()

price_comparison = [current_avg_price, recent_avg_price]
price_labels = ['Current Avg Price', '30-Day Avg Price']
bars = ax7.bar(price_labels, price_comparison, color=['#2D6A4F', '#52B788'])
ax7.set_title('Average Unit Price Comparison', fontweight='bold')
ax7.set_ylabel('Price ($)')
for bar, value in zip(bars, price_comparison):
    height = bar.get_height()
    ax7.text(bar.get_x() + bar.get_width()/2., height + max(price_comparison)*0.01, 
             f'${value:.2f}', ha='center', va='bottom', fontweight='bold')

# Price range analysis
ax8 = fig.add_subplot(gs[2, 2])
price_ranges = sku_enriched['max_unit_price'] - sku_enriched['min_unit_price']
ax8.hist(price_ranges, bins=20, color='#95D5B2', alpha=0.7, edgecolor='black')
ax8.set_title('Price Range Distribution', fontweight='bold')
ax8.set_xlabel('Price Range ($)')
ax8.set_ylabel('Number of Products')
ax8.axvline(price_ranges.mean(), color='red', linestyle='--', linewidth=2, 
           label=f'Mean: ${price_ranges.mean():.2f}')
ax8.legend()

plt.suptitle('Revenue Analysis Dashboard', fontsize=18, fontweight='bold', y=0.98)
plt.show()

# Print revenue insights
print(f"\n💸 KEY REVENUE INSIGHTS:")
print(f"• Total revenue: ${sku_enriched['total_revenue'].sum():,.2f}")
print(f"• 30-day revenue: ${sku_enriched['revenue_last_30d'].sum():,.2f}")
print(f"• Average revenue per product (30d): ${sku_enriched['revenue_last_30d'].mean():,.2f}")
print(f"• Top revenue product: {sku_enriched.loc[sku_enriched['revenue_last_30d'].idxmax(), 'name']}")
print(f"• Revenue concentration: Top 20% products generate {cumulative_pct.iloc[int(len(cumulative_pct)*0.2)-1]:.1f}% of revenue")
print(f"• Average unit price: ${sku_enriched['unit_price'].mean():.2f}")
print(f"• Price range (avg): ${price_ranges.mean():.2f}")


In [ ]:
# 7. Cash Flow & Working Capital Analysis
print("💰 CASH FLOW & WORKING CAPITAL ANALYSIS")
print("="*50)

# Calculate working capital metrics
def calculate_working_capital_metrics(df):
    """
    Calculate key working capital metrics for credit assessment
    """
    # Days Sales Outstanding (Revenue tied up in inventory)
    df['inventory_value'] = df['current_inventory'] * df['avg_unit_price']
    df['daily_revenue'] = df['revenue_last_30d'] / 30
    df['days_sales_outstanding'] = df['inventory_value'] / df['daily_revenue'].replace(0, np.nan)
    
    # Working capital efficiency
    df['revenue_per_inventory_dollar'] = df['revenue_last_30d'] / df['inventory_value'].replace(0, np.nan)
    
    # Cash generation velocity
    df['cash_velocity'] = df['revenue_last_30d'] / (df['inventory_value'] + 1)  # Add 1 to avoid division by zero
    
    return df

# Apply working capital calculations
sku_working_capital = calculate_working_capital_metrics(sku_enriched.copy())

# Create working capital dashboard
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Working Capital & Cash Flow Analysis - Private Credit Assessment', fontsize=16, fontweight='bold')

# 1. Days Sales Outstanding Distribution
dso_data = sku_working_capital['days_sales_outstanding'].dropna()
dso_data = dso_data[dso_data < dso_data.quantile(0.95)]  # Remove outliers
ax1.hist(dso_data, bins=30, color='#3498DB', alpha=0.7, edgecolor='black')
ax1.set_title('Days Sales Outstanding Distribution', fontweight='bold')
ax1.set_xlabel('Days')
ax1.set_ylabel('Number of Products')
ax1.axvline(dso_data.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {dso_data.mean():.1f} days')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Revenue per Inventory Dollar
rev_per_inv = sku_working_capital['revenue_per_inventory_dollar'].dropna()
rev_per_inv = rev_per_inv[rev_per_inv < rev_per_inv.quantile(0.95)]  # Remove outliers
ax2.hist(rev_per_inv, bins=30, color='#2ECC71', alpha=0.7, edgecolor='black')
ax2.set_title('Revenue per Inventory Dollar Efficiency', fontweight='bold')
ax2.set_xlabel('Revenue per $ of Inventory')
ax2.set_ylabel('Number of Products')
ax2.axvline(rev_per_inv.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ${rev_per_inv.mean():.2f}')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Cash Velocity vs Sales Performance
ax3.scatter(sku_working_capital['cash_velocity'], sku_working_capital['sales_last_30d'], 
           alpha=0.6, color='#E74C3C', s=50)
ax3.set_title('Cash Velocity vs Sales Performance', fontweight='bold')
ax3.set_xlabel('Cash Velocity (Revenue/Inventory Value)')
ax3.set_ylabel('Sales (30d)')
ax3.grid(True, alpha=0.3)

# 4. Working Capital Risk Matrix
# High inventory value + Low revenue = High risk
inv_value_norm = (sku_working_capital['inventory_value'] - sku_working_capital['inventory_value'].min()) / (sku_working_capital['inventory_value'].max() - sku_working_capital['inventory_value'].min())
revenue_norm = (sku_working_capital['revenue_last_30d'] - sku_working_capital['revenue_last_30d'].min()) / (sku_working_capital['revenue_last_30d'].max() - sku_working_capital['revenue_last_30d'].min())

scatter = ax4.scatter(inv_value_norm, revenue_norm, 
                     c=sku_working_capital['days_sales_outstanding'], 
                     cmap='RdYlBu_r', alpha=0.7, s=60)
ax4.set_title('Working Capital Risk Matrix', fontweight='bold')
ax4.set_xlabel('Inventory Value (Normalized)')
ax4.set_ylabel('Revenue Performance (Normalized)')
ax4.grid(True, alpha=0.3)

# Add risk quadrants
ax4.axhline(y=0.5, color='black', linestyle='--', alpha=0.5)
ax4.axvline(x=0.5, color='black', linestyle='--', alpha=0.5)
ax4.text(0.75, 0.25, 'High Risk\n(High Inv, Low Rev)', ha='center', va='center', 
         bbox=dict(boxstyle='round', facecolor='red', alpha=0.3))
ax4.text(0.25, 0.75, 'Low Risk\n(Low Inv, High Rev)', ha='center', va='center',
         bbox=dict(boxstyle='round', facecolor='green', alpha=0.3))

plt.colorbar(scatter, ax=ax4, label='Days Sales Outstanding')
plt.tight_layout()
plt.show()

# Print working capital insights
print("\n💰 WORKING CAPITAL INSIGHTS:")
print(f"• Average Days Sales Outstanding: {dso_data.mean():.1f} days")
print(f"• Average Revenue per Inventory $: ${rev_per_inv.mean():.2f}")
print(f"• Total inventory value: ${sku_working_capital['inventory_value'].sum():,.0f}")
print(f"• High risk products (top 25% DSO): {(sku_working_capital['days_sales_outstanding'] > sku_working_capital['days_sales_outstanding'].quantile(0.75)).sum()}")
